In [1]:
import numpy as np
import pandas as pd
import pyodbc as pyo
import sqlalchemy as sa
import numpy as np
import math
import datetime
import datetime as dt
#Kết nối SV chính (SV 65):
connection_uri = (
    "mssql+pyodbc://PHLHCMHOPPBI02\PBIRS/PowerBI?driver=ODBC+Driver+17+for+SQL+Server"
)
conn = sa.create_engine(connection_uri)
#Kết nối SV_UAT (SV 64):
connection_uri_UAT = (
    "mssql+pyodbc://PHLHCMHOUPBI01\PBIRS/PowerBI?driver=ODBC+Driver+17+for+SQL+Server"
)
conn_UAT = sa.create_engine(connection_uri_UAT)

<h3> 1. Lấy bảng

<h5> 1.1 Lấy bảng PY2

In [2]:
cutoff = '20230131'
period = cutoff[:6]

In [3]:
py2_query = f"SELECT * FROM [DPO].[DP_K2_CUTOFF] WHERE CUTOFF = {cutoff} " 
py2 = pd.read_sql(py2_query, conn)

<h5> 1.2 Lấy bảng Cấu trúc

In [4]:
structure_query = f"SELECT * FROM [dbo].[BSP_TAN_AGENT_STRUCTURE] WHERE CUTOFF = {cutoff}"
structure = pd.read_sql(structure_query ,conn_UAT)

<h5> 1.3 Lấy bảng KPI cá nhân

In [5]:
individual_kpi_query = f"SELECT * FROM [DPO].[BSP_TAN_INDIVIDUAL_KPI] WHERE Period = {period}"
individual_kpi = pd.read_sql(individual_kpi_query ,conn_UAT)

<h3>2. Tính PY2 

<h6>(Do PY2 cá nhân ko thể group by lại, nên phải tính theo từng cá nhân để group by lại )

In [6]:
py2_t = py2
py2_t = py2_t.groupby(['AGENT NO'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum()
py2_t

,Y2 ACTUAL PREM,Y2 EXPECTED PREM
AGENT NO,,
60000450,111587000.0,111587000.0
60000580,4236000.0,8437000.0
60000597,0.0,22714000.0
60001375,0.0,17860000.0
60001457,28758000.0,34098000.0
...,...,...
69999896,41397000.0,69193000.0
69999897,0.0,16314000.0
69999899,8015000.0,23108000.0


In [7]:
kpi_t = individual_kpi
kpi_t = pd.merge(kpi_t, py2_t, left_on ='Agent_Number', right_on = 'AGENT NO', how = 'left')
kpi_t = kpi_t.loc[:, kpi_t.columns != 'AGENT NO']
kpi_t

,Period,Agent_Number,Agent_Name,Grade,Agent_Status,Date_Appointed,Segmentation,Rookie,PY2,CC(+),...,AFYP(+),AFYP(-),AFYP,FYP(-),FYP(+),FYP,RYP,Case Size,Y2 ACTUAL PREM,Y2 EXPECTED PREM
0,202301,60060890,NGUYỄN HOÀNG ANH,FC,Enforce,2022-06-20,Agent Trainee,,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,202301,60060917,PHẠM BÁ KHƯƠNG,FC,Terminated,2022-06-20,Agent Trainee,,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,202301,60021398,NGUYỄN THỊ MẬU,FC,Terminated,2019-06-19,FC,,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,202301,60055999,NGUYỄN THỊ BIÊN,FC,Enforce,2022-01-25,FC,,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,202301,60075691,LÊ THỊ KHÁNH LY,FC,Enforce,2022-12-29,Agent Trainee,60,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75901,202301,60006002,NGÔ QUỐC TRUNG,FC,Terminated,2016-09-26,FC,,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
75902,202301,60074423,PHAN THỊ TUYẾT HƯƠNG,FC,Enforce,2022-12-12,Agent Trainee,60,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
75903,202301,60016239,LÊ THỊ PHƯỚC,FC,Terminated,2018-08-24,FC,,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
75904,202301,60005479,HỒ NGỌC THÔNG,FC,Terminated,2016-08-25,FC,,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN


<h5> 2.1 Join để lấy PY2 nhóm

<h6> 2.1.1 FM Team

In [8]:
FM_team = structure
FM_team = FM_team[FM_team['FM_Code'] != '0']
FM_team = FM_team[['FM_Code', 'FM_Name', 'AgentNumber', 'AgentName', 'AgentGrade']].drop_duplicates()

In [9]:
#Tính PY2 từ đây
FM_team_py2 = pd.merge(FM_team, kpi_t[['Agent_Number', 'Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']], left_on = 'AgentNumber', right_on = 'Agent_Number', how = 'left' )
FM_team_py2 = FM_team_py2.groupby(['FM_Code', 'FM_Name'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum()
FM_team_py2['PY2'] = np.where(FM_team_py2['Y2 EXPECTED PREM'] == 0, 0, FM_team_py2['Y2 ACTUAL PREM']/FM_team_py2['Y2 EXPECTED PREM'])
FM_team_py2['PY2'] = np.where(FM_team_py2['PY2'] > 1, 1, FM_team_py2['PY2'])
FM_team_py2 = FM_team_py2.reset_index()

In [10]:
FM_team_py2 = FM_team_py2[['FM_Code', 'FM_Name', 'PY2']]
FM_team_py2['Grade'] = "FM"
FM_team_py2.rename(columns={'FM_Code':'AL_Code', 'FM_Name':'AL_Name'}, inplace = True)
FM_team_py2

,AL_Code,AL_Name,PY2,Grade
0,60000014,PHAN THANH THẢO,0.0,FM
1,60000037,NGÔ VĂN TOÁN,0.0,FM
2,60000055,ĐINH HỒ QUANG VINH,0.0,FM
3,60000100,NGUYỄN VĂN SỨ,0.0,FM
4,60000101,BÙI THỊ HƯỜNG,0.0,FM
...,...,...,...,...
8058,60076154,ĐỖ THỊ CHÂM,0.0,FM
8059,60076155,NGUYỄN THỊ AN,0.0,FM
8060,60076158,DUMMY PHÙNG THỊ LUYẾN,0.0,FM
8061,60076161,DUMMY HOÀNG THỊ HÀ,0.0,FM


<h5> 2.1.2 DM Team 

In [11]:
DM_team = structure
DM_team = DM_team[DM_team['DM_Code'] != '0']
DM_team = DM_team[['DM_Code', 'DM_Name', 'AgentNumber', 'AgentName', 'AgentGrade']].drop_duplicates()
#Tính PY2 từ đây
DM_team_py2 = pd.merge(DM_team, kpi_t[['Agent_Number', 'Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']], left_on = 'AgentNumber', right_on = 'Agent_Number', how = 'left' )
DM_team_py2 = DM_team_py2.groupby(['DM_Code', 'DM_Name'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum()
DM_team_py2['PY2'] = np.where(DM_team_py2['Y2 EXPECTED PREM'] == 0, 0, DM_team_py2['Y2 ACTUAL PREM']/DM_team_py2['Y2 EXPECTED PREM'])
DM_team_py2['PY2'] = np.where(DM_team_py2['PY2'] > 1, 1, DM_team_py2['PY2'])
DM_team_py2 = DM_team_py2.reset_index()
DM_team_py2 = DM_team_py2[['DM_Code', 'DM_Name', 'PY2']]
DM_team_py2['Grade'] = "DM"
DM_team_py2.rename(columns={'DM_Code':'AL_Code', 'DM_Name':'AL_Name'}, inplace = True)
DM_team_py2

,AL_Code,AL_Name,PY2,Grade
0,60000092,NGUYỄN THỊ THU,0.0,DM
1,60000231,TRƯƠNG NGUYỄN MẠNH CƯƠNG,0.0,DM
2,60000256,TRẦN QUYẾT THẮNG,0.0,DM
3,60000263,VÕ ĐOÀN PHƯƠNG THÚY,0.0,DM
4,60000459,ĐẶNG THỊ HOÀNG YẾN,0.0,DM
...,...,...,...,...
2157,60076129,ĐỚI THỊ HUỆ,0.0,DM
2158,60076133,DUMMY NGUYỄN THỊ ĐIỀN,0.0,DM
2159,60076138,DUMMY NGUYỄN THỊ LOAN,0.0,DM
2160,60118829,TRỊNH ĐẠI NGHĨA,0.0,DM


<h5> 2.1.3 RM Team

In [12]:
RM_team = structure
RM_team = RM_team[RM_team['RM_Code'] != '0']
RM_team = RM_team[['RM_Code', 'RM_Name', 'AgentNumber', 'AgentName', 'AgentGrade']].drop_duplicates()
#Tính PY2 từ đây
RM_team_py2 = pd.merge(RM_team, kpi_t[['Agent_Number', 'Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']], left_on = 'AgentNumber', right_on = 'Agent_Number', how = 'left' )
RM_team_py2 = RM_team_py2.groupby(['RM_Code', 'RM_Name'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum()
RM_team_py2['PY2'] = np.where(RM_team_py2['Y2 EXPECTED PREM'] == 0, 0, RM_team_py2['Y2 ACTUAL PREM']/RM_team_py2['Y2 EXPECTED PREM'])
RM_team_py2['PY2'] = np.where(RM_team_py2['PY2'] > 1, 1, RM_team_py2['PY2'])
RM_team_py2 = RM_team_py2.reset_index()
RM_team_py2 = RM_team_py2[['RM_Code', 'RM_Name', 'PY2']]
RM_team_py2['Grade'] = "RM"
RM_team_py2.rename(columns={'RM_Code':'AL_Code', 'RM_Name':'AL_Name'}, inplace = True)
RM_team_py2

,AL_Code,AL_Name,PY2,Grade
0,60000727,NGUYỄN THỊ NGỌC HIỀN,0.309324,RM
1,60005252,TRẦN THỊ THÙY VÂN,0.000000,RM
2,60012146,PHAN THỊ LỆ HẰNG,0.542641,RM
3,60012256,LÊ VĂN TUYẾN,0.626865,RM
4,60014549,NGUYỄN CÔNG DŨNG,0.741198,RM
...,...,...,...,...
73,60058061,NGUYỄN THỊ LỆ TUYẾT,0.000000,RM
74,60059003,LÊ CHÍ TRUNG,0.000000,RM
75,60060164,LÂM TUẤN KIỆT,0.000000,RM
76,60061923,PHẠM TÀI LỘC,0.000000,RM


<h5>2.1.4 GM Team

In [13]:
GM_team = structure
GM_team = GM_team[GM_team['GM_Code'] != '0']
GM_team = GM_team[['GM_Code', 'GM_Name', 'AgentNumber', 'AgentName', 'AgentGrade']].drop_duplicates()
#Tính PY2 từ đây
GM_team_py2 = pd.merge(GM_team, kpi_t[['Agent_Number', 'Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']], left_on = 'AgentNumber', right_on = 'Agent_Number', how = 'left' )
GM_team_py2 = GM_team_py2.groupby(['GM_Code', 'GM_Name'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum()
GM_team_py2['PY2'] = np.where(GM_team_py2['Y2 EXPECTED PREM'] == 0, 0, GM_team_py2['Y2 ACTUAL PREM']/GM_team_py2['Y2 EXPECTED PREM'])
GM_team_py2['PY2'] = np.where(GM_team_py2['PY2'] > 1, 1, GM_team_py2['PY2'])
GM_team_py2 = GM_team_py2.reset_index()
GM_team_py2 = GM_team_py2[['GM_Code', 'GM_Name', 'PY2']]
GM_team_py2['Grade'] = "GM"
GM_team_py2.rename(columns={'GM_Code':'AL_Code', 'GM_Name':'AL_Name'}, inplace = True)
GM_team_py2

,AL_Code,AL_Name,PY2,Grade
0,60011794,LÊ THỊ THU HẰNG,0.614745,GM
1,60023377,HỒ THỊ HÀ,0.000000,GM
2,60023547,NGUYỄN ĐĂNG TIẾN,0.390879,GM


<h5> 2.1.5 Tổng hợp PY2

In [14]:
PY2_list = pd.concat([GM_team_py2, RM_team_py2, DM_team_py2, FM_team_py2])
PY2_list['PY2'] = round(PY2_list['PY2'], 4)
PY2_list = PY2_list[['AL_Code', 'AL_Name' ,'PY2']]
PY2_list

,AL_Code,AL_Name,PY2
0,60011794,LÊ THỊ THU HẰNG,0.6147
1,60023377,HỒ THỊ HÀ,0.0000
2,60023547,NGUYỄN ĐĂNG TIẾN,0.3909
0,60000727,NGUYỄN THỊ NGỌC HIỀN,0.3093
1,60005252,TRẦN THỊ THÙY VÂN,0.0000
...,...,...,...
8058,60076154,ĐỖ THỊ CHÂM,0.0000
8059,60076155,NGUYỄN THỊ AN,0.0000
8060,60076158,DUMMY PHÙNG THỊ LUYẾN,0.0000
8061,60076161,DUMMY HOÀNG THỊ HÀ,0.0000


<h3>3. Tính Man Power

In [15]:
manpower = structure
manpower.head()

,Area_Name,Sales_Unit_Code,Sales_Unit,GM_Whole_Code,GM_Whole_Name,GM_IndCode,GM_IndName,GM_Code,GM_Name,RM_Whole_Code,...,Hand_Phone,Date_Appointed,Terminated_date,Promote_Date,Demote_Date,TERRITORY,Leader,Parent_Supervisor_Code,Supervisor_Code,CUTOFF
0,TAA,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,0783972109,2022-06-20,0,0,0,0,60030655,60030650,60030655,20230131
1,ABT,AD786,Nguyễn Thị Tố Quyên,0,0,0,0,0,0,60055383,...,0947233487,2022-06-20,2022-10-19,0,0,0,60055914,60055833,60055914,20230131
2,EAK,AD174,Thiều Quang Mạnh,0,0,0,0,0,0,0,...,0395726395,2019-06-19,2020-03-16,0,0,0,60008686,60008686,60019636,20230131
3,AVP,AD811,Văn Hương Giang,0,0,0,0,0,0,0,...,0392472170,2022-01-25,0,0,0,0,60054777,,60054777,20230131
4,LAK,AD501,Đặng Văn Phan,0,0,0,0,0,0,60021241,...,0818319419,2022-12-29,0,0,0,0,60075163,60062231,60075163,20230131


<h5>3.1 Lấy ra ngày xét trong tháng 

In [16]:
manpower['Date_On_Set'] = "" #Tạo ra cột trống
manpower['Date_On_Set']= manpower['Date_On_Set'].str.cat(manpower['CUTOFF'].str[:4] ,sep ="")
manpower['Date_On_Set']= manpower['Date_On_Set'].str.cat(manpower['CUTOFF'].str[4:6] ,sep ="-")
manpower['Date_On_Set']= manpower['Date_On_Set'].str.cat(manpower['CUTOFF'].str[6:8] ,sep ="-")
manpower['Date_On_Set'] = pd.to_datetime(manpower['Date_On_Set']) #nhớ chuyển sang kiểu ngày tháng
if ('20230531' in manpower['CUTOFF'].values):
    manpower.loc[:,'Start_Month'] = '2023-04-30'
elif ('20220228' in manpower['CUTOFF'].values):
    manpower.loc[:,'Start_Month'] = '2022-01-28'
elif ('20220429' in manpower['CUTOFF'].values):
    manpower.loc[:,'Start_Month'] = '2022-03-31'
elif ('20220531' in manpower['CUTOFF'].values):
    manpower.loc[:,'Start_Month'] = '2022-04-30' 
elif ('20240229' in manpower['CUTOFF'].values):
    manpower.loc[:,'Start_Month'] = '2024-01-27'
else: 
    manpower['Start_Month'] = (manpower['Date_On_Set'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))
manpower

,Area_Name,Sales_Unit_Code,Sales_Unit,GM_Whole_Code,GM_Whole_Name,GM_IndCode,GM_IndName,GM_Code,GM_Name,RM_Whole_Code,...,Terminated_date,Promote_Date,Demote_Date,TERRITORY,Leader,Parent_Supervisor_Code,Supervisor_Code,CUTOFF,Date_On_Set,Start_Month
0,TAA,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,0,0,0,0,60030655,60030650,60030655,20230131,2023-01-31,2023-01-01
1,ABT,AD786,Nguyễn Thị Tố Quyên,0,0,0,0,0,0,60055383,...,2022-10-19,0,0,0,60055914,60055833,60055914,20230131,2023-01-31,2023-01-01
2,EAK,AD174,Thiều Quang Mạnh,0,0,0,0,0,0,0,...,2020-03-16,0,0,0,60008686,60008686,60019636,20230131,2023-01-31,2023-01-01
3,AVP,AD811,Văn Hương Giang,0,0,0,0,0,0,0,...,0,0,0,0,60054777,,60054777,20230131,2023-01-31,2023-01-01
4,LAK,AD501,Đặng Văn Phan,0,0,0,0,0,0,60021241,...,0,0,0,0,60075163,60062231,60075163,20230131,2023-01-31,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75901,SG3,AD246,Nguyễn Thanh Lộc,0,0,0,0,0,0,0,...,2022-08-17,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75902,CLA,AD285,Nguyễn Ánh Tuyết,0,0,0,0,0,0,0,...,0,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75903,VIN,AD158,Nguyễn Thị Nguyệt,0,0,0,0,0,0,0,...,2022-10-27,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75904,SG1,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,0,0,0,0,0,0,,20230131,2023-01-31,2023-01-01


<h5> 3.2 Tính Inforce các cấp

In [17]:
inforce = manpower
inforce

,Area_Name,Sales_Unit_Code,Sales_Unit,GM_Whole_Code,GM_Whole_Name,GM_IndCode,GM_IndName,GM_Code,GM_Name,RM_Whole_Code,...,Terminated_date,Promote_Date,Demote_Date,TERRITORY,Leader,Parent_Supervisor_Code,Supervisor_Code,CUTOFF,Date_On_Set,Start_Month
0,TAA,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,0,0,0,0,60030655,60030650,60030655,20230131,2023-01-31,2023-01-01
1,ABT,AD786,Nguyễn Thị Tố Quyên,0,0,0,0,0,0,60055383,...,2022-10-19,0,0,0,60055914,60055833,60055914,20230131,2023-01-31,2023-01-01
2,EAK,AD174,Thiều Quang Mạnh,0,0,0,0,0,0,0,...,2020-03-16,0,0,0,60008686,60008686,60019636,20230131,2023-01-31,2023-01-01
3,AVP,AD811,Văn Hương Giang,0,0,0,0,0,0,0,...,0,0,0,0,60054777,,60054777,20230131,2023-01-31,2023-01-01
4,LAK,AD501,Đặng Văn Phan,0,0,0,0,0,0,60021241,...,0,0,0,0,60075163,60062231,60075163,20230131,2023-01-31,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75901,SG3,AD246,Nguyễn Thanh Lộc,0,0,0,0,0,0,0,...,2022-08-17,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75902,CLA,AD285,Nguyễn Ánh Tuyết,0,0,0,0,0,0,0,...,0,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75903,VIN,AD158,Nguyễn Thị Nguyệt,0,0,0,0,0,0,0,...,2022-10-27,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75904,SG1,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,0,0,0,0,0,0,,20230131,2023-01-31,2023-01-01


<h6> 3.2.1 FM

In [18]:
fm_inforce = inforce
fm_inforce = fm_inforce[['FM_Code', 'FM_Name', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status']]
fm_inforce = fm_inforce[(fm_inforce['Agent_Status'] !='Terminated') & (fm_inforce['FM_Code'] != '0')]
fm_inforce = pd.merge(fm_inforce, kpi_t[['Agent_Number', 'Segmentation']], left_on = 'AgentNumber', right_on='Agent_Number', how ='left')
fm_inforce = fm_inforce.loc[:,fm_inforce.columns != 'Agent_Number']
fm_inforce

,FM_Code,FM_Name,AgentNumber,AgentName,AgentGrade,Agent_Status,Segmentation
0,60030655,PHẠM THỊ HỒNG ANH,60060890,NGUYỄN HOÀNG ANH,FC,Enforce,Agent Trainee
1,60075163,ĐẶNG THỊ SOA,60075691,LÊ THỊ KHÁNH LY,FC,Enforce,Agent Trainee
2,60045563,KSOR MAM,60056723,SIU PHOAR,FC,Enforce,FC
3,60015691,TRẦN THỊ HƯƠNG,60025077,THÁI THỊ THÚY,FC,Enforce,SFC
4,60045525,NGUYỄN VĂN TIỀN,60053620,NGUYỄN THỊ THANH LAN,FC,Enforce,SFC
...,...,...,...,...,...,...,...
20842,60044241,ĐẶNG TRƯƠNG TRỌNG,60044241,ĐẶNG TRƯƠNG TRỌNG,FM,Enforce,AL
20843,60034611,HỒ NGUYỄN ĐÍNH,60034611,HỒ NGUYỄN ĐÍNH,FM,Enforce,AL
20844,60030184,BÙI THÁI TRỌNG,60030184,BÙI THÁI TRỌNG,FM,Enforce,AL
20845,60032521,NGUYỄN TẤN ĐẠT,60032521,NGUYỄN TẤN ĐẠT,FM,Enforce,AL


In [19]:
segment = fm_inforce['Segmentation'].drop_duplicates().to_list()
segment

['Agent Trainee', 'FC', 'SFC', 'AL', 'AL Trainee']

<h3> ###Lưu ý cách Pivot kết quả <h3>

In [144]:
#fm_inforce_sum = fm_inforce
#fm_inforce_sum = fm_inforce_sum.groupby(['FM_Code','FM_Name', 'Segmentation']).size().reset_index(name='count')
#fm_inforce_sum

,FM_Code,FM_Name,Segmentation,count
0,60000482,NGUYỄN THỊ NƯƠNG,AL,1
1,60000482,NGUYỄN THỊ NƯƠNG,Agent Trainee,1
2,60000482,NGUYỄN THỊ NƯƠNG,FC,7
3,60000717,NGUYỄN THỊ KIM YẾN,AL,1
4,60000717,NGUYỄN THỊ KIM YẾN,FC,6
...,...,...,...,...
9384,60076155,NGUYỄN THỊ AN,AL,1
9385,60076155,NGUYỄN THỊ AN,FC,2
9386,60076158,DUMMY PHÙNG THỊ LUYẾN,AL Trainee,1
9387,60076161,DUMMY HOÀNG THỊ HÀ,AL Trainee,1


In [20]:
fm_inforce_sum = fm_inforce
fm_inforce_sum = fm_inforce_sum.groupby(['FM_Code','FM_Name','Segmentation']).size().unstack(fill_value=0)
fm_inforce_sum = fm_inforce_sum.reset_index()
fm_inforce_sum['Grade'] = "FM"
fm_inforce_sum = fm_inforce_sum.rename(columns={'FM_Code': 'AL_Code', 'FM_Name': 'AL_Name'})
fm_inforce_sum

Segmentation,AL_Code,AL_Name,AL,AL Trainee,Agent Trainee,FC,SFC,Grade
0,60000482,NGUYỄN THỊ NƯƠNG,1,0,1,7,0,FM
1,60000717,NGUYỄN THỊ KIM YẾN,1,0,0,6,0,FM
2,60001826,CAO NGỌC DUY DANH,1,0,0,0,1,FM
3,60002523,NGUYỄN THỊ THANH NHÀN,1,0,0,3,11,FM
4,60005359,TRẦN THỊ TUYẾT,1,0,0,2,9,FM
...,...,...,...,...,...,...,...,...
4659,60076153,DUMMY NGUYỄN CHÍNH PHÓNG,0,1,0,0,0,FM
4660,60076154,ĐỖ THỊ CHÂM,1,0,0,2,0,FM
4661,60076155,NGUYỄN THỊ AN,1,0,0,2,0,FM
4662,60076158,DUMMY PHÙNG THỊ LUYẾN,0,1,0,0,0,FM


<h6>3.2.2 DM

In [21]:
DM_inforce = inforce
DM_inforce = DM_inforce[['DM_Code', 'DM_Name', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status']]
DM_inforce = DM_inforce[(DM_inforce['Agent_Status'] !='Terminated') & (DM_inforce['DM_Code'] != '0')]
DM_inforce = pd.merge(DM_inforce, kpi_t[['Agent_Number', 'Segmentation']], left_on = 'AgentNumber', right_on='Agent_Number', how ='left')
DM_inforce = DM_inforce.loc[:,DM_inforce.columns != 'Agent_Number']
#
DM_inforce_sum = DM_inforce
DM_inforce_sum = DM_inforce_sum.groupby(['DM_Code','DM_Name', 'Segmentation']).size().unstack(fill_value=0)
DM_inforce_sum = DM_inforce_sum.reset_index()
DM_inforce_sum['Grade'] = "DM"
DM_inforce_sum = DM_inforce_sum.rename(columns={'DM_Code': 'AL_Code', 'DM_Name': 'AL_Name'})
DM_inforce_sum

Segmentation,AL_Code,AL_Name,AL,AL Trainee,Agent Trainee,FC,PHL,SFC,Grade
0,60001255,NGUYỄN HỒNG TƯƠI,5,0,0,24,0,13,DM
1,60002588,DOÃN TIẾN AN,3,0,0,3,0,0,DM
2,60003349,PHẠM THỊ MỸ HẠNH,7,0,0,6,0,5,DM
3,60003396,NGUYỄN THANH HÙNG,5,0,0,18,0,0,DM
4,60004324,NGUYỄN THỊ ĐÔNG,1,0,0,0,0,0,DM
...,...,...,...,...,...,...,...,...,...
1201,60076129,ĐỚI THỊ HUỆ,3,0,0,4,0,0,DM
1202,60076133,DUMMY NGUYỄN THỊ ĐIỀN,0,1,0,0,0,0,DM
1203,60076138,DUMMY NGUYỄN THỊ LOAN,0,1,0,0,0,0,DM
1204,60118829,TRỊNH ĐẠI NGHĨA,4,0,0,3,0,0,DM


<h6> 3.2.3 RM

In [22]:
RM_inforce = inforce
RM_inforce = RM_inforce[['RM_Code', 'RM_Name', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status']]
RM_inforce = RM_inforce[(RM_inforce['Agent_Status'] !='Terminated') & (RM_inforce['RM_Code'] != '0')]
RM_inforce = pd.merge(RM_inforce, kpi_t[['Agent_Number', 'Segmentation']], left_on = 'AgentNumber', right_on='Agent_Number', how ='left')
RM_inforce = RM_inforce.loc[:,RM_inforce.columns != 'Agent_Number']
#
RM_inforce_sum = RM_inforce
RM_inforce_sum = RM_inforce_sum.groupby(['RM_Code','RM_Name', 'Segmentation']).size().unstack(fill_value=0)
RM_inforce_sum = RM_inforce_sum.reset_index()
RM_inforce_sum['Grade'] = "RM"
RM_inforce_sum = RM_inforce_sum.rename(columns={'RM_Code': 'AL_Code', 'RM_Name': 'AL_Name'})
RM_inforce_sum

Segmentation,AL_Code,AL_Name,AL,AL Trainee,Agent Trainee,FC,SFC,Grade
0,60000727,NGUYỄN THỊ NGỌC HIỀN,23,0,11,77,38,RM
1,60005252,TRẦN THỊ THÙY VÂN,9,0,235,29,1,RM
2,60012146,PHAN THỊ LỆ HẰNG,21,0,0,61,56,RM
3,60012256,LÊ VĂN TUYẾN,35,0,3,107,67,RM
4,60014549,NGUYỄN CÔNG DŨNG,20,0,0,183,77,RM
5,60014821,TRƯƠNG HỒNG HÀ,22,0,0,161,147,RM
6,60014874,NGUYỄN HOÀNG DŨNG,34,1,2,127,18,RM
7,60014977,TRẦN NỮ TUYẾT SƯƠNG,24,0,5,94,32,RM
8,60020362,ĐỖ THỊ LOAN,13,0,6,65,50,RM
9,60020387,NGUYỄN THỊ ANH TRÂM,30,0,0,63,43,RM


<h6>3.2.4 GM

In [23]:
GM_inforce = inforce
GM_inforce = GM_inforce[['GM_Code', 'GM_Name', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status']]
GM_inforce = GM_inforce[(GM_inforce['Agent_Status'] !='Terminated') & (GM_inforce['GM_Code'] != '0')]
GM_inforce = pd.merge(GM_inforce, kpi_t[['Agent_Number', 'Segmentation']], left_on = 'AgentNumber', right_on='Agent_Number', how ='left')
GM_inforce = GM_inforce.loc[:,GM_inforce.columns != 'Agent_Number']
#
GM_inforce_sum = GM_inforce
GM_inforce_sum = GM_inforce_sum.groupby(['GM_Code','GM_Name', 'Segmentation']).size().unstack(fill_value=0)
GM_inforce_sum = GM_inforce_sum.reset_index()
GM_inforce_sum['Grade'] = "GM"
GM_inforce_sum = GM_inforce_sum.rename(columns={'GM_Code': 'AL_Code', 'GM_Name': 'AL_Name'})
GM_inforce_sum

Segmentation,AL_Code,AL_Name,AL,AL Trainee,Agent Trainee,FC,SFC,Grade
0,60011794,LÊ THỊ THU HẰNG,62,0,3,191,130,GM
1,60023547,NGUYỄN ĐĂNG TIẾN,43,1,2,361,113,GM


<h4> 3.3 Tổng hợp lại ManPower Inforce (Cấp AL, Agent)

In [24]:
Manpower_Inforce = pd.concat([fm_inforce_sum, DM_inforce_sum, RM_inforce_sum, GM_inforce_sum ])
Manpower_Inforce

Segmentation,AL_Code,AL_Name,AL,AL Trainee,Agent Trainee,FC,SFC,Grade,PHL
0,60000482,NGUYỄN THỊ NƯƠNG,1,0,1,7,0,FM,NaN
1,60000717,NGUYỄN THỊ KIM YẾN,1,0,0,6,0,FM,NaN
2,60001826,CAO NGỌC DUY DANH,1,0,0,0,1,FM,NaN
3,60002523,NGUYỄN THỊ THANH NHÀN,1,0,0,3,11,FM,NaN
4,60005359,TRẦN THỊ TUYẾT,1,0,0,2,9,FM,NaN
...,...,...,...,...,...,...,...,...,...
57,60059003,LÊ CHÍ TRUNG,9,1,209,20,0,RM,NaN
58,60060164,LÂM TUẤN KIỆT,2,0,1,3,0,RM,NaN
59,60061923,PHẠM TÀI LỘC,11,0,178,26,0,RM,NaN
0,60011794,LÊ THỊ THU HẰNG,62,0,3,191,130,GM,NaN


<h4> 3.4 Tổng lấy GM -> FM 

In [25]:
AL_inforce = inforce
AL_inforce

,Area_Name,Sales_Unit_Code,Sales_Unit,GM_Whole_Code,GM_Whole_Name,GM_IndCode,GM_IndName,GM_Code,GM_Name,RM_Whole_Code,...,Terminated_date,Promote_Date,Demote_Date,TERRITORY,Leader,Parent_Supervisor_Code,Supervisor_Code,CUTOFF,Date_On_Set,Start_Month
0,TAA,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,0,0,0,0,60030655,60030650,60030655,20230131,2023-01-31,2023-01-01
1,ABT,AD786,Nguyễn Thị Tố Quyên,0,0,0,0,0,0,60055383,...,2022-10-19,0,0,0,60055914,60055833,60055914,20230131,2023-01-31,2023-01-01
2,EAK,AD174,Thiều Quang Mạnh,0,0,0,0,0,0,0,...,2020-03-16,0,0,0,60008686,60008686,60019636,20230131,2023-01-31,2023-01-01
3,AVP,AD811,Văn Hương Giang,0,0,0,0,0,0,0,...,0,0,0,0,60054777,,60054777,20230131,2023-01-31,2023-01-01
4,LAK,AD501,Đặng Văn Phan,0,0,0,0,0,0,60021241,...,0,0,0,0,60075163,60062231,60075163,20230131,2023-01-31,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75901,SG3,AD246,Nguyễn Thanh Lộc,0,0,0,0,0,0,0,...,2022-08-17,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75902,CLA,AD285,Nguyễn Ánh Tuyết,0,0,0,0,0,0,0,...,0,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75903,VIN,AD158,Nguyễn Thị Nguyệt,0,0,0,0,0,0,0,...,2022-10-27,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75904,SG1,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,0,0,0,0,0,0,,20230131,2023-01-31,2023-01-01


<h6> 3.4.1 Lấy FM:

In [26]:
AL_lst = ['FM', 'DM', 'RM', 'GM']

<h5> ### Lưu ý cách thêm 1 cột khi chưa có

In [27]:
FM_count = AL_inforce[['FM_Code', 'FM_Name', 'AgentGrade']]
FM_count = FM_count[FM_count['FM_Code'] != '0']
FM_count = FM_count.groupby(['FM_Code', 'FM_Name', 'AgentGrade']).size().unstack(fill_value=0)
FM_count = FM_count.reset_index()
for col in AL_lst:
    if col not in FM_count.columns:
        FM_count[col] = 0
FM_count = FM_count.rename(columns={'FM_Code':'AL_Code', 'FM_Name': 'AL_Name'})
FM_count['Grade'] = 'FM'
FM_count

AgentGrade,AL_Code,AL_Name,DM,FC,FM,RM,GM,Grade
0,60000014,PHAN THANH THẢO,0,0,1,0,0,FM
1,60000037,NGÔ VĂN TOÁN,0,1,1,0,0,FM
2,60000055,ĐINH HỒ QUANG VINH,0,1,1,0,0,FM
3,60000100,NGUYỄN VĂN SỨ,0,0,1,0,0,FM
4,60000101,BÙI THỊ HƯỜNG,0,4,1,0,0,FM
...,...,...,...,...,...,...,...,...
8058,60076154,ĐỖ THỊ CHÂM,0,2,1,0,0,FM
8059,60076155,NGUYỄN THỊ AN,0,2,1,0,0,FM
8060,60076158,DUMMY PHÙNG THỊ LUYẾN,0,0,1,0,0,FM
8061,60076161,DUMMY HOÀNG THỊ HÀ,0,1,1,0,0,FM


<h5> 3.4.2 Lấy DM:

In [28]:
DM_count = AL_inforce[['DM_Code', 'DM_Name', 'AgentGrade']]
DM_count = DM_count[DM_count['DM_Code'] != '0']
DM_count = DM_count.groupby(['DM_Code', 'DM_Name', 'AgentGrade']).size().unstack(fill_value=0)
DM_count = DM_count.reset_index()
for col in AL_lst:
    if col not in DM_count.columns:
        DM_count[col] = 0
DM_count = DM_count.rename(columns={'DM_Code':'AL_Code', 'DM_Name': 'AL_Name'})
DM_count['Grade'] = 'DM'
DM_count

AgentGrade,AL_Code,AL_Name,DM,FC,FM,RM,GM,Grade
0,60000092,NGUYỄN THỊ THU,1,13,1,0,0,DM
1,60000231,TRƯƠNG NGUYỄN MẠNH CƯƠNG,1,4,3,0,0,DM
2,60000256,TRẦN QUYẾT THẮNG,1,1,0,0,0,DM
3,60000263,VÕ ĐOÀN PHƯƠNG THÚY,1,0,1,0,0,DM
4,60000459,ĐẶNG THỊ HOÀNG YẾN,1,0,0,0,0,DM
...,...,...,...,...,...,...,...,...
2157,60076129,ĐỚI THỊ HUỆ,1,4,2,0,0,DM
2158,60076133,DUMMY NGUYỄN THỊ ĐIỀN,1,0,0,0,0,DM
2159,60076138,DUMMY NGUYỄN THỊ LOAN,1,0,0,0,0,DM
2160,60118829,TRỊNH ĐẠI NGHĨA,1,3,3,0,0,DM


<h5> 3.4.3 Lấy RM:

In [29]:
RM_count = AL_inforce[['RM_Code', 'RM_Name', 'AgentGrade']]
RM_count = RM_count[RM_count['RM_Code'] != '0']
RM_count = RM_count.groupby(['RM_Code', 'RM_Name', 'AgentGrade']).size().unstack(fill_value=0)
RM_count = RM_count.reset_index()
for col in AL_lst:
    if col not in RM_count.columns:
        RM_count[col] = 0
RM_count = RM_count.rename(columns={'RM_Code':'AL_Code', 'RM_Name': 'AL_Name'})
RM_count['Grade'] = 'RM'
RM_count

AgentGrade,AL_Code,AL_Name,DM,FC,FM,RM,GM,Grade
0,60000727,NGUYỄN THỊ NGỌC HIỀN,3,281,20,1,0,RM
1,60005252,TRẦN THỊ THÙY VÂN,2,344,6,1,0,RM
2,60012146,PHAN THỊ LỆ HẰNG,4,277,39,1,0,RM
3,60012256,LÊ VĂN TUYẾN,5,367,29,1,0,RM
4,60014549,NGUYỄN CÔNG DŨNG,3,645,19,1,0,RM
...,...,...,...,...,...,...,...,...
73,60058061,NGUYỄN THỊ LỆ TUYẾT,3,75,13,1,0,RM
74,60059003,LÊ CHÍ TRUNG,3,287,6,1,0,RM
75,60060164,LÂM TUẤN KIỆT,0,49,1,1,0,RM
76,60061923,PHẠM TÀI LỘC,4,232,6,1,0,RM


<h5> 3.4.4 Lấy GM

In [30]:
GM_count = AL_inforce[['GM_Code', 'GM_Name', 'AgentGrade']]
GM_count = GM_count[GM_count['GM_Code'] != '0']
GM_count = GM_count.groupby(['GM_Code', 'GM_Name', 'AgentGrade']).size().unstack(fill_value=0)
GM_count = GM_count.reset_index()
for col in AL_lst:
    if col not in GM_count.columns:
        GM_count[col] = 0
GM_count = GM_count.rename(columns={'GM_Code':'AL_Code', 'GM_Name': 'AL_Name'})
GM_count['Grade'] = 'GM'
GM_count

AgentGrade,AL_Code,AL_Name,DM,FC,FM,GM,RM,Grade
0,60011794,LÊ THỊ THU HẰNG,12,605,48,1,2,GM
1,60023377,HỒ THỊ HÀ,0,15,0,1,0,GM
2,60023547,NGUYỄN ĐĂNG TIẾN,8,635,42,1,4,GM


<h4> 3.5 Hợp nhất lấy GM -> FM

In [31]:
GM_FM = pd.concat([FM_count, DM_count, RM_count, GM_count])
GM_FM

AgentGrade,AL_Code,AL_Name,DM,FC,FM,RM,GM,Grade
0,60000014,PHAN THANH THẢO,0,0,1,0,0,FM
1,60000037,NGÔ VĂN TOÁN,0,1,1,0,0,FM
2,60000055,ĐINH HỒ QUANG VINH,0,1,1,0,0,FM
3,60000100,NGUYỄN VĂN SỨ,0,0,1,0,0,FM
4,60000101,BÙI THỊ HƯỜNG,0,4,1,0,0,FM
...,...,...,...,...,...,...,...,...
76,60061923,PHẠM TÀI LỘC,4,232,6,1,0,RM
77,60071896,CHÂU THỊ NGUYỆT,1,2,0,1,0,RM
0,60011794,LÊ THỊ THU HẰNG,12,605,48,2,1,GM
1,60023377,HỒ THỊ HÀ,0,15,0,0,1,GM


<h4> 3.6 ManPower Inforce hoàn chỉnh

In [32]:
Inforce_MP = pd.merge(Manpower_Inforce, GM_FM, on = ['AL_Code', 'AL_Name'], how = 'left')
Inforce_MP = Inforce_MP[['AL_Code', 'AL_Name', 'AL', 'AL Trainee', 'Agent Trainee', 'FC_x', 'SFC', 'Grade_x', 'PHL', 'FM', 'DM', 'RM', 'GM']].\
rename(columns={'Grade_x':'Grade', 'FC_x':'FC'})
Inforce_MP

,AL_Code,AL_Name,AL,AL Trainee,Agent Trainee,FC,SFC,Grade,PHL,FM,DM,RM,GM
0,60000482,NGUYỄN THỊ NƯƠNG,1,0,1,7,0,FM,NaN,1,0,0,0
1,60000717,NGUYỄN THỊ KIM YẾN,1,0,0,6,0,FM,NaN,1,0,0,0
2,60001826,CAO NGỌC DUY DANH,1,0,0,0,1,FM,NaN,1,0,0,0
3,60002523,NGUYỄN THỊ THANH NHÀN,1,0,0,3,11,FM,NaN,1,0,0,0
4,60005359,TRẦN THỊ TUYẾT,1,0,0,2,9,FM,NaN,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5927,60059003,LÊ CHÍ TRUNG,9,1,209,20,0,RM,NaN,6,3,1,0
5928,60060164,LÂM TUẤN KIỆT,2,0,1,3,0,RM,NaN,1,0,1,0
5929,60061923,PHẠM TÀI LỘC,11,0,178,26,0,RM,NaN,6,4,1,0
5930,60011794,LÊ THỊ THU HẰNG,62,0,3,191,130,GM,NaN,48,12,2,1


<h3> 4. Tính Recruitment

In [33]:
recruitment = manpower
recruitment = recruitment[(recruitment['Date_Appointed'] >= recruitment['Start_Month']) & 
                         (recruitment['Date_Appointed'] <= recruitment['Date_On_Set'])]
recruitment

,Area_Name,Sales_Unit_Code,Sales_Unit,GM_Whole_Code,GM_Whole_Name,GM_IndCode,GM_IndName,GM_Code,GM_Name,RM_Whole_Code,...,Terminated_date,Promote_Date,Demote_Date,TERRITORY,Leader,Parent_Supervisor_Code,Supervisor_Code,CUTOFF,Date_On_Set,Start_Month
274,QNH,AD943,Phạm Hòa,0,0,0,0,0,0,0,...,0,0,0,0,60075990,,60075990,20230131,2023-01-31,2023-01-01
307,AHD,AD931,AG Lê Thị Hường,0,0,0,0,0,0,0,...,0,0,0,0,60075182,60075254,60075182,20230131,2023-01-31,2023-01-01
387,LDG,AD262,Bùi Minh Thư,0,0,0,0,0,0,0,...,0,0,0,0,60075946,,60075946,20230131,2023-01-31,2023-01-01
415,CTO,AD961,Hồ Thị Hồng Mai,0,0,0,0,0,0,0,...,0,0,0,0,60075249,60075249,60075573,20230131,2023-01-31,2023-01-01
701,HN1,AD892,Vũ Khắc Hiệp,0,0,0,0,0,0,0,...,0,0,0,0,60071977,60071926,60071977,20230131,2023-01-31,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74807,ALX,AD990,AG Huỳnh Trọng Nghĩa,0,0,0,0,0,0,0,...,0,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75333,GLA,AD278,Nguyễn Phi Hùng,0,0,0,0,0,0,0,...,0,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75697,ABT,AD991,Nguyễn Thị Thùy Linh,0,0,0,0,0,0,0,...,0,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75749,DGH,AD988,Nguyễn Hoàng Hiệp,0,0,0,0,0,0,0,...,0,0,0,0,0,0,,20230131,2023-01-31,2023-01-01


<h5> 4.1 Tính FM

In [34]:
FM_recruit = recruitment
FM_recruit = FM_recruit[['FM_Code', 'FM_Name', 'AgentGrade']]
FM_recruit = FM_recruit[FM_recruit['FM_Code'] != '0']
FM_recruit = FM_recruit.groupby(['FM_Code', 'FM_Name', 'AgentGrade']).size().unstack(fill_value=0)
for col in AL_lst:
    if col not in FM_recruit.columns:
        FM_recruit[col] = 0
FM_recruit = FM_recruit.reset_index()
FM_recruit['Grade'] = "FM"
FM_recruit = FM_recruit.rename(columns={'FM_Code': 'AL_Code', 'FM_Name':'AL_Name'})
FM_recruit

AgentGrade,AL_Code,AL_Name,FC,FM,DM,RM,GM,Grade
0,60005807,NGUYỄN THANH LONG,2,1,0,0,0,FM
1,60008924,PHÙNG THỊ PHƯỢNG,2,0,0,0,0,FM
2,60019397,PHẠM THỊ NGỌC MAI THẢO,2,1,0,0,0,FM
3,60029956,DUMMY NGUYỄN VĂN LINH,1,0,0,0,0,FM
4,60038082,PHAN THỊ BIỂN,1,1,0,0,0,FM
...,...,...,...,...,...,...,...,...
192,60076153,DUMMY NGUYỄN CHÍNH PHÓNG,0,1,0,0,0,FM
193,60076154,ĐỖ THỊ CHÂM,2,1,0,0,0,FM
194,60076155,NGUYỄN THỊ AN,2,1,0,0,0,FM
195,60076158,DUMMY PHÙNG THỊ LUYẾN,0,1,0,0,0,FM


<h5>4.2 Tính DM

In [35]:
DM_recruit = recruitment
DM_recruit = DM_recruit[['DM_Code', 'DM_Name', 'AgentGrade']]
DM_recruit = DM_recruit[DM_recruit['DM_Code'] != '0']
DM_recruit = DM_recruit.groupby(['DM_Code', 'DM_Name', 'AgentGrade']).size().unstack(fill_value=0)
for col in AL_lst:
    if col not in DM_recruit.columns:
        DM_recruit[col] = 0
DM_recruit = DM_recruit.reset_index()
DM_recruit['Grade'] = "DM"
DM_recruit = DM_recruit.rename(columns={'DM_Code': 'AL_Code', 'DM_Name':'AL_Name'})
DM_recruit

AgentGrade,AL_Code,AL_Name,DM,FC,FM,RM,GM,Grade
0,60006365,VŨ XUÂN THÀNH,0,2,0,0,0,DM
1,60007436,PHẠM KIỀU NỮ BÍCH QUYÊN,1,3,1,0,0,DM
2,60012548,QUÁCH VĂN DỰ,0,1,0,0,0,DM
3,60016289,NGUYỄN TIẾN BÍNH,0,1,0,0,0,DM
4,60020931,LÊ MINH THỌ,1,4,1,0,0,DM
...,...,...,...,...,...,...,...,...
168,60076107,DUMMY VŨ VĂN HẢI,1,3,1,0,0,DM
169,60076109,NGUYỄN THỊ CẨM HẰNG,1,2,2,0,0,DM
170,60076129,ĐỚI THỊ HUỆ,1,4,2,0,0,DM
171,60076133,DUMMY NGUYỄN THỊ ĐIỀN,1,0,0,0,0,DM


<h5>4.3 Tính RM

In [36]:
RM_recruit = recruitment
RM_recruit = RM_recruit[['RM_Code', 'RM_Name', 'AgentGrade']]
RM_recruit = RM_recruit[RM_recruit['RM_Code'] != '0']
RM_recruit = RM_recruit.groupby(['RM_Code', 'RM_Name', 'AgentGrade']).size().unstack(fill_value=0)
for col in AL_lst:
    if col not in RM_recruit.columns:
        RM_recruit[col] = 0
RM_recruit = RM_recruit.reset_index()
RM_recruit['Grade'] = "RM"
RM_recruit = RM_recruit.rename(columns={'RM_Code': 'AL_Code', 'RM_Name':'AL_Name'})
RM_recruit

AgentGrade,AL_Code,AL_Name,DM,FC,FM,RM,GM,Grade
0,60012256,LÊ VĂN TUYẾN,0,3,0,0,0,RM
1,60014874,NGUYỄN HOÀNG DŨNG,0,1,0,0,0,RM
2,60022528,VŨ THỊ MÙI,0,3,0,0,0,RM
3,60033716,NGUYỄN NGỌC ÁNH,0,1,0,0,0,RM
4,60037558,NGUYỄN TRÍ HÙNG,0,9,6,0,0,RM
5,60039194,LÊ THỊ THÙY TRANG,0,1,0,0,0,RM
6,60041229,HOÀNG TUYẾT NHUNG,0,6,1,0,0,RM
7,60046604,PHẠM MINH SÁNG,0,2,1,0,0,RM
8,60053282,ĐỚI SỸ HÀ,1,7,3,0,0,RM
9,60055392,ĐỖ THỊ THÚY MAI,0,0,1,0,0,RM


<h5>4.4 Tính GM

In [37]:
GM_recruit = recruitment
GM_recruit = GM_recruit[['GM_Code', 'GM_Name', 'AgentGrade']]
GM_recruit = GM_recruit[GM_recruit['GM_Code'] != '0']
GM_recruit = GM_recruit.groupby(['GM_Code', 'GM_Name', 'AgentGrade']).size().unstack(fill_value=0)
for col in AL_lst:
    if col not in GM_recruit.columns:
        GM_recruit[col] = 0
GM_recruit = GM_recruit.reset_index()
GM_recruit['Grade'] = "GM"
GM_recruit = GM_recruit.rename(columns={'GM_Code': 'AL_Code', 'GM_Name':'AL_Name'})
GM_recruit

AgentGrade,AL_Code,AL_Name,FC,FM,DM,RM,GM,Grade
0,60011794,LÊ THỊ THU HẰNG,3,0,0,0,0,GM


<h5> 4.5 Tổng hợp tuyển dụng trong tháng

In [38]:
monthly_recruit = pd.concat([FM_recruit, DM_recruit, RM_recruit, GM_recruit])
monthly_recruit

AgentGrade,AL_Code,AL_Name,FC,FM,DM,RM,GM,Grade
0,60005807,NGUYỄN THANH LONG,2,1,0,0,0,FM
1,60008924,PHÙNG THỊ PHƯỢNG,2,0,0,0,0,FM
2,60019397,PHẠM THỊ NGỌC MAI THẢO,2,1,0,0,0,FM
3,60029956,DUMMY NGUYỄN VĂN LINH,1,0,0,0,0,FM
4,60038082,PHAN THỊ BIỂN,1,1,0,0,0,FM
...,...,...,...,...,...,...,...,...
12,60057293,VŨ THỊ VÒNG,4,0,0,0,0,RM
13,60057831,NGUYỄN THỊ THANH HẢI,1,1,0,0,0,RM
14,60059003,LÊ CHÍ TRUNG,5,1,0,0,0,RM
15,60061923,PHẠM TÀI LỘC,0,1,0,0,0,RM


<h5> 4.6 Tính Recruit Active

In [39]:
recruit_active = pd.merge(recruitment, individual_kpi[['Agent_Number', 'Active Agent']], left_on ='AgentNumber', right_on = 'Agent_Number', how ='left')
recruit_active = recruit_active[recruit_active['Active Agent'] == '1']
recruit_active = recruit_active.loc[:, recruit_active.columns != 'Agent_Number']
recruit_active

,Area_Name,Sales_Unit_Code,Sales_Unit,GM_Whole_Code,GM_Whole_Name,GM_IndCode,GM_IndName,GM_Code,GM_Name,RM_Whole_Code,...,Promote_Date,Demote_Date,TERRITORY,Leader,Parent_Supervisor_Code,Supervisor_Code,CUTOFF,Date_On_Set,Start_Month,Active Agent
4,HN1,AD892,Vũ Khắc Hiệp,0,0,0,0,0,0,0,...,0,0,0,60071977,60071926,60071977,20230131,2023-01-31,2023-01-01,1
7,AHP,AD748,AG Nguyễn Đình Nghiệp,0,0,0,0,0,0,0,...,0,0,0,60075786,60075786,60075787,20230131,2023-01-31,2023-01-01,1
10,THO,AD630,Nguyễn Duy Hùng,0,0,0,0,0,0,60052560,...,0,0,0,60076119,60076119,60076117,20230131,2023-01-31,2023-01-01,1
13,BDU,AD602,Quách Cẩm Chương,0,0,0,0,0,0,0,...,0,0,0,60054127,60054126,60054127,20230131,2023-01-31,2023-01-01,1
14,DNA,AD800,Lý Đăng Khoa,0,0,0,0,0,0,0,...,0,0,0,60075332,60074534,60075332,20230131,2023-01-31,2023-01-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,DN1,AD795,Nguyễn Thị Phương Lan,0,0,0,0,0,0,0,...,0,0,0,60075281,60075280,60075281,20230131,2023-01-31,2023-01-01,1
416,THO,AD751,Lê Thị Hiếu,0,0,0,0,0,0,0,...,0,0,0,0,,60059940,20230131,2023-01-31,2023-01-01,1
417,AHD,AD773,Đồng Thế Huy,0,0,0,0,0,0,0,...,0,0,0,60076107,,60076107,20230131,2023-01-31,2023-01-01,1
422,ACM,AD807,AG Trần Thanh Miền,0,0,0,0,0,0,0,...,0,0,0,60067591,60067591,60067593,20230131,2023-01-31,2023-01-01,1


In [40]:
#Lấy ra FM:
FM_active = recruit_active
FM_active = FM_active[['FM_Code', 'AgentNumber', 'Active Agent']]
FM_active = FM_active[FM_active['FM_Code'] != '0']
FM_active = FM_active.groupby(['FM_Code'])['Active Agent'].count()
FM_active = FM_active.reset_index()
FM_active.rename(columns={'FM_Code':'AL_Code'}, inplace = True)
FM_active

,AL_Code,Active Agent
0,60038082,2
1,60038836,1
2,60055379,1
3,60059146,2
4,60059147,2
...,...,...
76,60076113,1
77,60076119,3
78,60076126,1
79,60076154,1


In [41]:
#Lấy ra DM:
DM_active = recruit_active
DM_active = DM_active[['DM_Code', 'AgentNumber', 'Active Agent']]
DM_active = DM_active[DM_active['DM_Code'] != '0']
DM_active = DM_active.groupby(['DM_Code'])['Active Agent'].count()
DM_active = DM_active.reset_index()
DM_active.rename(columns={'DM_Code':'AL_Code'}, inplace = True)
DM_active

,AL_Code,Active Agent
0,60006365,1
1,60020931,1
2,60033294,2
3,60038825,1
4,60053985,1
...,...,...
68,60075885,4
69,60075971,2
70,60075990,1
71,60076107,4


In [42]:
#Lấy ra RM:
RM_active = recruit_active
RM_active = RM_active[['RM_Code', 'AgentNumber', 'Active Agent']]
RM_active = RM_active[RM_active['RM_Code'] != '0']
RM_active = RM_active.groupby(['RM_Code'])['Active Agent'].count()
RM_active = RM_active.reset_index()
RM_active.rename(columns={'RM_Code':'AL_Code'}, inplace = True)
RM_active

,AL_Code,Active Agent
0,60012256,1
1,60033716,1
2,60037558,8
3,60041229,2
4,60053282,6
5,60055392,1
6,60056663,2
7,60057293,4
8,60059003,4


In [43]:
#Lấy ra GM:
GM_active = recruit_active
GM_active = GM_active[['GM_Code', 'AgentNumber', 'Active Agent']]
GM_active = GM_active[GM_active['GM_Code'] != '0']
GM_active = GM_active.groupby(['GM_Code'])['Active Agent'].count()
GM_active = GM_active.reset_index()
GM_active.rename(columns={'GM_Code':'AL_Code'}, inplace = True)
GM_active

,AL_Code,Active Agent
0,60011794,1


In [44]:
active_list = pd.concat([FM_active, DM_active, RM_active, GM_active])
active_list.rename(columns={'Active Agent': 'Recruit Active'}, inplace = True)
active_list                       

,AL_Code,Recruit Active
0,60038082,2
1,60038836,1
2,60055379,1
3,60059146,2
4,60059147,2
...,...,...
5,60055392,1
6,60056663,2
7,60057293,4
8,60059003,4


<h5> 4.7 Tính Tổng các chỉ số Recruitment

In [45]:
recruitment_full = pd.merge(monthly_recruit, active_list, on='AL_Code', how = 'left')
recruitment_full.fillna(0, inplace = True)
for col in ['FC', 'FM','DM','RM','GM','Recruit Active']:
    recruitment_full[col] = recruitment_full[col].astype('int')
recruitment_full['Recruit AL'] =0 
for col in ['FC', 'FM','DM','RM','GM']:
    recruitment_full['Recruit AL'] += recruitment_full[col]
recruitment_full.rename(columns={'FC': 'Recruit FC'}, inplace =True)
recruitment_full['Recruit Total'] = recruitment_full['Recruit FC'] + recruitment_full['Recruit AL']
recruitment_full['ActiveRatioNewRecruit'] = np.where(recruitment_full['Recruit Total'] == 0,0,
                                    round(recruitment_full['Recruit Active'] / recruitment_full['Recruit Total'], 4))
for i in ['FM', 'DM', 'RM', 'GM']:
    recruitment_full.rename(columns={i:'Recruit_'+i}, inplace=True)
recruitment_full

,AL_Code,AL_Name,Recruit FC,Recruit_FM,Recruit_DM,Recruit_RM,Recruit_GM,Grade,Recruit Active,Recruit AL,Recruit Total,ActiveRatioNewRecruit
0,60005807,NGUYỄN THANH LONG,2,1,0,0,0,FM,0,3,5,0.0000
1,60008924,PHÙNG THỊ PHƯỢNG,2,0,0,0,0,FM,0,2,4,0.0000
2,60019397,PHẠM THỊ NGỌC MAI THẢO,2,1,0,0,0,FM,0,3,5,0.0000
3,60029956,DUMMY NGUYỄN VĂN LINH,1,0,0,0,0,FM,0,1,2,0.0000
4,60038082,PHAN THỊ BIỂN,1,1,0,0,0,FM,2,2,3,0.6667
...,...,...,...,...,...,...,...,...,...,...,...,...
382,60057293,VŨ THỊ VÒNG,4,0,0,0,0,RM,4,4,8,0.5000
383,60057831,NGUYỄN THỊ THANH HẢI,1,1,0,0,0,RM,0,2,3,0.0000
384,60059003,LÊ CHÍ TRUNG,5,1,0,0,0,RM,4,6,11,0.3636
385,60061923,PHẠM TÀI LỘC,0,1,0,0,0,RM,0,1,1,0.0000


<h3>5. Tính Terminate

In [46]:
terminate_active = manpower
terminate_active = terminate_active[terminate_active['Terminated_date'] != '0']
terminate_active['Terminated_date'] = pd.to_datetime(terminate_active['Terminated_date'])
terminate_active = terminate_active[(terminate_active['Terminated_date'] >= terminate_active['Start_Month']) &
                   (terminate_active['Terminated_date'] <= terminate_active['Date_On_Set']) ]
terminate_agent = terminate_active
terminate_agent['AgentGrade'] = np.where(terminate_agent['AgentGrade'].isin(['FC', 'FM', 'DM', 'RM', 'GM']), terminate_agent['AgentGrade'],
                                        'FC')
terminate_agent

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_14556\588302934.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  terminate_active['Terminated_date'] = pd.to_datetime(terminate_active['Terminated_date'])


,Area_Name,Sales_Unit_Code,Sales_Unit,GM_Whole_Code,GM_Whole_Name,GM_IndCode,GM_IndName,GM_Code,GM_Name,RM_Whole_Code,...,Terminated_date,Promote_Date,Demote_Date,TERRITORY,Leader,Parent_Supervisor_Code,Supervisor_Code,CUTOFF,Date_On_Set,Start_Month
104,BRI,AD441,Nguyễn Thị Thanh Thúy,0,0,0,0,0,0,0,...,2023-01-13,0,0,0,60008930,60046890,60038479,20230131,2023-01-31,2023-01-01
755,HUE,AD291,Phan Thị Xuân,0,0,0,0,0,0,0,...,2023-01-31,0,0,0,60043903,60024554,60043903,20230131,2023-01-31,2023-01-01
1676,BRI,AD441,Nguyễn Thị Thanh Thúy,0,0,0,0,0,0,0,...,2023-01-13,0,0,0,60008930,60046890,60044677,20230131,2023-01-31,2023-01-01
1855,BH2,AD712,Nguyễn Thị Hòa,0,0,0,0,0,0,0,...,2023-01-12,0,0,0,60069432,60057723,60069432,20230131,2023-01-31,2023-01-01
3432,VI2,AD717,Nguyễn Thị Vân Nga,0,0,0,0,0,0,0,...,2023-01-05,0,0,0,60069113,60068617,60069113,20230131,2023-01-31,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74854,TTU,AD068,Công ty Thiên Tú,0,0,0,0,0,0,0,...,2023-01-10,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75549,VI2,AD539,Cao Thị Thúy,0,0,0,0,0,0,0,...,2023-01-31,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75610,ADN,AD859,Nguyễn Thị Ước Thành,0,0,0,0,0,0,0,...,2023-01-31,0,0,0,0,0,,20230131,2023-01-31,2023-01-01
75736,DSR,AD097,Nguyễn Hữu Thọ,0,0,0,0,0,0,0,...,2023-01-10,0,0,0,0,0,,20230131,2023-01-31,2023-01-01


In [47]:
#Tính FM:
FM_ter = terminate_agent
FM_ter = FM_ter[['FM_Code', 'AgentNumber', 'AgentGrade']]
FM_ter= FM_ter[FM_ter['FM_Code'] != '0']
FM_ter['AL Ter'] = np.where(FM_ter['AgentGrade'] == 'FC', 0, 1)
FM_ter['FC Ter'] = np.where(FM_ter['AgentGrade'] == 'FC', 1, 0)
FM_ter = FM_ter.groupby(['FM_Code'])[['AL Ter', 'FC Ter']].sum().reset_index().rename(columns={'FM_Code': 'AL_Code'})
FM_ter

,AL_Code,AL Ter,FC Ter
0,60008930,0,4
1,60021639,0,1
2,60022354,0,1
3,60025280,0,2
4,60029166,0,1
...,...,...,...
83,60069432,1,1
84,60070443,0,1
85,60072717,1,0
86,60073147,0,1


In [48]:
#Tính DM:
DM_ter = terminate_agent
DM_ter = DM_ter[['DM_Code', 'AgentNumber', 'AgentGrade']]
DM_ter= DM_ter[DM_ter['DM_Code'] != '0']
DM_ter['AL Ter'] = np.where(DM_ter['AgentGrade'] == 'FC', 0, 1)
DM_ter['FC Ter'] = np.where(DM_ter['AgentGrade'] == 'FC', 1, 0)
DM_ter = DM_ter.groupby(['DM_Code'])[['AL Ter', 'FC Ter']].sum().reset_index().rename(columns={'DM_Code': 'AL_Code'})
DM_ter

,AL_Code,AL Ter,FC Ter
0,60005229,0,1
1,60007708,0,1
2,60009490,0,1
3,60011097,0,1
4,60021548,0,1
...,...,...,...
72,60067268,0,1
73,60067662,0,1
74,60069081,1,0
75,60072456,1,0


In [49]:
#Tính RM:
RM_ter = terminate_agent
RM_ter = RM_ter[['RM_Code', 'AgentNumber', 'AgentGrade']]
RM_ter= RM_ter[RM_ter['RM_Code'] != '0']
RM_ter['AL Ter'] = np.where(RM_ter['AgentGrade'] == 'FC', 0, 1)
RM_ter['FC Ter'] = np.where(RM_ter['AgentGrade'] == 'FC', 1, 0)
RM_ter = RM_ter.groupby(['RM_Code'])[['AL Ter', 'FC Ter']].sum().reset_index().rename(columns={'RM_Code': 'AL_Code'})
RM_ter

,AL_Code,AL Ter,FC Ter
0,60020387,0,1
1,60027069,0,9
2,60027824,0,1
3,60029603,2,0
4,60031416,1,0
5,60037503,0,1
6,60037558,1,0
7,60041788,1,0
8,60041797,1,1
9,60058061,1,1


In [50]:
#Tính GM:
GM_ter = terminate_agent
GM_ter = GM_ter[['GM_Code', 'AgentNumber', 'AgentGrade']]
GM_ter= GM_ter[GM_ter['GM_Code'] != '0']
GM_ter['AL Ter'] = np.where(GM_ter['AgentGrade'] == 'FC', 0, 1)
GM_ter['FC Ter'] = np.where(GM_ter['AgentGrade'] == 'FC', 1, 0)
GM_ter = GM_ter.groupby(['GM_Code'])[['AL Ter', 'FC Ter']].sum().reset_index().rename(columns={'GM_Code': 'AL_Code'})
GM_ter

,AL_Code,AL Ter,FC Ter
0,60011794,0,1


<h3> 5.1 Danh sách TER

In [51]:
ter_list = pd.concat([FM_ter, DM_ter, RM_ter, GM_ter])
ter_list

,AL_Code,AL Ter,FC Ter
0,60008930,0,4
1,60021639,0,1
2,60022354,0,1
3,60025280,0,2
4,60029166,0,1
...,...,...,...
6,60037558,1,0
7,60041788,1,0
8,60041797,1,1
9,60058061,1,1


<h3> 6. Tổng hợp giai đoạn I

In [52]:
agent_info = structure
agent_info = agent_info[agent_info['AgentGrade'].isin(['FM', 'DM', 'RM', 'GM'])]
agent_info['Period'] = agent_info['CUTOFF'].str[:6]
agent_info = agent_info[['Period', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status', 'Date_Appointed']]
agent_info

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_14556\396033571.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_info['Period'] = agent_info['CUTOFF'].str[:6]


,Period,AgentNumber,AgentName,AgentGrade,Agent_Status,Date_Appointed
37,202301,60054915,MAI THẾ TRUNG,FM,Enforce,2021-12-24
44,202301,60006025,NGUYỄN THỊ HẠNH,FM,Terminated,2016-09-26
52,202301,60013413,NGÔ THỦY TRIỀU,FM,Terminated,2018-03-26
65,202301,60056899,DUMMY HỒ THỊ LONG,FM,Terminated,2022-03-03
67,202301,60037327,DUMMY 37327,FM,Terminated,2021-01-07
...,...,...,...,...,...,...
75891,202301,60038668,LƯƠNG THỊ TƯỜNG VI,DM,Enforce,2021-03-03
75892,202301,60033477,MÃ THÀNH HIẾU,DM,Enforce,2022-01-25
75894,202301,60045450,LƯƠNG ANH QUÂN,FM,Enforce,2021-06-23
75895,202301,60008668,NGUYỄN THỊ SÁU,DM,Terminated,2017-06-23


<h2> LƯU Ý CÁCH BỎ CỘT KHÔNG CẦN THIẾT VỚI ~

In [53]:
#PY2:
al_py2 = pd.merge(agent_info, PY2_list, left_on='AgentNumber', right_on='AL_Code', how= 'left')
al_py2 = al_py2[['Period', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status','Date_Appointed', 'PY2']]
al_py2
#MP:
al_mp = pd.merge(al_py2, Inforce_MP, left_on='AgentNumber', right_on='AL_Code', how= 'left' )
al_mp = al_mp.loc[:, ~al_mp.columns.isin(['AL_Code','AL_Name'])]
al_mp['MP_Total'] = al_mp['AL'] + al_mp['FC']
al_mp = al_mp[['Period', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status','Date_Appointed', 'PY2',
        'GM', 'RM', 'DM', 'FM', 'AL', 'FC', 'MP_Total', 'AL Trainee', 'Agent Trainee', 'SFC']]
al_mp
#Recruit:
recruit_al = pd.merge(al_mp, recruitment_full, left_on='AgentNumber', right_on='AL_Code', how= 'left')
recruit_al = recruit_al.loc[:,~recruit_al.columns.isin(['Grade', 'AL_Code', 'AL_Name'])]
recruit_al
#ter:
ter_al = pd.merge(recruit_al, ter_list, left_on='AgentNumber', right_on='AL_Code', how= 'left')
ter_al = ter_al.loc[:, ~ter_al.columns.isin(['AL_Code'])]
ter_al['Total Agent Ter'] = ter_al['AL Ter'] + ter_al['FC Ter']
ter_al['%Ter'] = np.where((ter_al['MP_Total'] + ter_al['Total Agent Ter'] ==0), 0, 
                round(ter_al['Total Agent Ter'] / (ter_al['MP_Total'] + ter_al['Total Agent Ter']), 4))
ter_al = ter_al[['Period', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status','Date_Appointed', 'PY2', 
        'GM', 'RM', 'DM', 'FM', 'AL', 'FC', 'MP_Total', 'AL Trainee', 'Agent Trainee', 'SFC',
        'Recruit_RM', 'Recruit_DM', 'Recruit_FM', 'Recruit AL', 'Recruit FC', 'Recruit Total', 'Recruit Active', 'ActiveRatioNewRecruit',
        'AL Ter','FC Ter', 'Total Agent Ter', '%Ter']]
ter_al

,Period,AgentNumber,AgentName,AgentGrade,Agent_Status,Date_Appointed,PY2,GM,RM,DM,...,Recruit_FM,Recruit AL,Recruit FC,Recruit Total,Recruit Active,ActiveRatioNewRecruit,AL Ter,FC Ter,Total Agent Ter,%Ter
0,202301,60054915,MAI THẾ TRUNG,FM,Enforce,2021-12-24,0.0000,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202301,60006025,NGUYỄN THỊ HẠNH,FM,Terminated,2016-09-26,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202301,60013413,NGÔ THỦY TRIỀU,FM,Terminated,2018-03-26,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202301,60056899,DUMMY HỒ THỊ LONG,FM,Terminated,2022-03-03,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202301,60037327,DUMMY 37327,FM,Terminated,2021-01-07,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10301,202301,60038668,LƯƠNG THỊ TƯỜNG VI,DM,Enforce,2021-03-03,0.0699,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10302,202301,60033477,MÃ THÀNH HIẾU,DM,Enforce,2022-01-25,0.6428,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10303,202301,60045450,LƯƠNG ANH QUÂN,FM,Enforce,2021-06-23,0.0000,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10304,202301,60008668,NGUYỄN THỊ SÁU,DM,Terminated,2017-06-23,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h3> 7. Tổng hợp giai đoạn 2

<h3>7.1 Lấy CaseCount

In [54]:
cc_kpi = individual_kpi
cc_kpi = cc_kpi[['Agent_Number', 'CC(+)', 'CC(-)', 'CC']]
for col in ['CC(+)', 'CC(-)', 'CC']:
    cc_kpi[col] = cc_kpi[col].astype('int')
cc_kpi = pd.merge(structure, cc_kpi, left_on = 'AgentNumber', right_on ='Agent_Number', how='left')
cc_kpi

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_14556\2784517125.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc_kpi[col] = cc_kpi[col].astype('int')
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_14556\2784517125.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc_kpi[col] = cc_kpi[col].astype('int')
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_14556\2784517125.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

,Area_Name,Sales_Unit_Code,Sales_Unit,GM_Whole_Code,GM_Whole_Name,GM_IndCode,GM_IndName,GM_Code,GM_Name,RM_Whole_Code,...,Leader,Parent_Supervisor_Code,Supervisor_Code,CUTOFF,Date_On_Set,Start_Month,Agent_Number,CC(+),CC(-),CC
0,TAA,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,60030655,60030650,60030655,20230131,2023-01-31,2023-01-01,60060890,0,0,0
1,ABT,AD786,Nguyễn Thị Tố Quyên,0,0,0,0,0,0,60055383,...,60055914,60055833,60055914,20230131,2023-01-31,2023-01-01,60060917,0,0,0
2,EAK,AD174,Thiều Quang Mạnh,0,0,0,0,0,0,0,...,60008686,60008686,60019636,20230131,2023-01-31,2023-01-01,60021398,0,0,0
3,AVP,AD811,Văn Hương Giang,0,0,0,0,0,0,0,...,60054777,,60054777,20230131,2023-01-31,2023-01-01,60055999,0,0,0
4,LAK,AD501,Đặng Văn Phan,0,0,0,0,0,0,60021241,...,60075163,60062231,60075163,20230131,2023-01-31,2023-01-01,60075691,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75901,SG3,AD246,Nguyễn Thanh Lộc,0,0,0,0,0,0,0,...,0,0,,20230131,2023-01-31,2023-01-01,60039816,0,0,0
75902,CLA,AD285,Nguyễn Ánh Tuyết,0,0,0,0,0,0,0,...,0,0,,20230131,2023-01-31,2023-01-01,60045618,0,0,0
75903,VIN,AD158,Nguyễn Thị Nguyệt,0,0,0,0,0,0,0,...,0,0,,20230131,2023-01-31,2023-01-01,60027162,0,0,0
75904,SG1,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,0,0,,20230131,2023-01-31,2023-01-01,60025232,0,0,0


In [55]:
#FM:
FM_cc = cc_kpi[['FM_Code', 'AgentNumber', 'CC(+)', 'CC(-)', 'CC' ]]
FM_cc = FM_cc[FM_cc['FM_Code'] != '0']
FM_cc = FM_cc.groupby(['FM_Code'])[['CC(+)', 'CC(-)', 'CC']].sum().reset_index()
FM_cc = FM_cc.rename(columns={'FM_Code':'AL_Code'})
FM_cc

,AL_Code,CC(+),CC(-),CC
0,60000014,0,0,0
1,60000037,0,0,0
2,60000055,0,0,0
3,60000100,0,0,0
4,60000101,0,0,0
...,...,...,...,...
8057,60076154,2,0,2
8058,60076155,2,0,2
8059,60076158,0,0,0
8060,60076161,0,0,0


In [56]:
#DM:
DM_cc = cc_kpi[['DM_Code', 'AgentNumber', 'CC(+)', 'CC(-)', 'CC' ]]
DM_cc = DM_cc[DM_cc['DM_Code'] != '0']
DM_cc = DM_cc.groupby(['DM_Code'])[['CC(+)', 'CC(-)', 'CC']].sum().reset_index()
DM_cc = DM_cc.rename(columns={'DM_Code':'AL_Code'})
DM_cc

,AL_Code,CC(+),CC(-),CC
0,60000092,0,0,0
1,60000231,0,0,0
2,60000256,0,0,0
3,60000263,0,0,0
4,60000459,0,0,0
...,...,...,...,...
2157,60076129,8,0,8
2158,60076133,0,0,0
2159,60076138,0,0,0
2160,60118829,1,0,1


In [57]:
#RM:
RM_cc = cc_kpi[['RM_Code', 'AgentNumber', 'CC(+)', 'CC(-)', 'CC' ]]
RM_cc = RM_cc[RM_cc['RM_Code'] != '0']
RM_cc = RM_cc.groupby(['RM_Code'])[['CC(+)', 'CC(-)', 'CC']].sum().reset_index()
RM_cc = RM_cc.rename(columns={'RM_Code':'AL_Code'})
RM_cc

,AL_Code,CC(+),CC(-),CC
0,60000727,8,0,8
1,60005252,21,-2,19
2,60012146,1,0,1
3,60012256,6,-4,2
4,60014549,3,-3,0
...,...,...,...,...
73,60058061,8,-1,7
74,60059003,20,0,20
75,60060164,0,0,0
76,60061923,26,0,26


In [58]:
#GM:
GM_cc = cc_kpi[['GM_Code', 'AgentNumber', 'CC(+)', 'CC(-)', 'CC' ]]
GM_cc = GM_cc[GM_cc['GM_Code'] != '0']
GM_cc = GM_cc.groupby(['GM_Code'])[['CC(+)', 'CC(-)', 'CC']].sum().reset_index()
GM_cc = GM_cc.rename(columns={'GM_Code':'AL_Code'})
GM_cc

,AL_Code,CC(+),CC(-),CC
0,60011794,7,-4,3
1,60023377,0,0,0
2,60023547,2,-7,-5


In [59]:
agent_cc = pd.concat([FM_cc, DM_cc, RM_cc, GM_cc])
agent_cc

,AL_Code,CC(+),CC(-),CC
0,60000014,0,0,0
1,60000037,0,0,0
2,60000055,0,0,0
3,60000100,0,0,0
4,60000101,0,0,0
...,...,...,...,...
76,60061923,26,0,26
77,60071896,0,0,0
0,60011794,7,-4,3
1,60023377,0,0,0


<h5> Ghép vào bảng trước đó

In [60]:
al_cc = pd.merge(ter_al, agent_cc, left_on='AgentNumber', right_on ='AL_Code', how = 'left')
al_cc = al_cc.loc[:, al_cc.columns != 'AL_Code']
al_cc

,Period,AgentNumber,AgentName,AgentGrade,Agent_Status,Date_Appointed,PY2,GM,RM,DM,...,Recruit Total,Recruit Active,ActiveRatioNewRecruit,AL Ter,FC Ter,Total Agent Ter,%Ter,CC(+),CC(-),CC
0,202301,60054915,MAI THẾ TRUNG,FM,Enforce,2021-12-24,0.0000,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,202301,60006025,NGUYỄN THỊ HẠNH,FM,Terminated,2016-09-26,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,202301,60013413,NGÔ THỦY TRIỀU,FM,Terminated,2018-03-26,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,202301,60056899,DUMMY HỒ THỊ LONG,FM,Terminated,2022-03-03,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,202301,60037327,DUMMY 37327,FM,Terminated,2021-01-07,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10301,202301,60038668,LƯƠNG THỊ TƯỜNG VI,DM,Enforce,2021-03-03,0.0699,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
10302,202301,60033477,MÃ THÀNH HIẾU,DM,Enforce,2022-01-25,0.6428,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
10303,202301,60045450,LƯƠNG ANH QUÂN,FM,Enforce,2021-06-23,0.0000,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
10304,202301,60008668,NGUYỄN THỊ SÁU,DM,Terminated,2017-06-23,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


<h3>7.2 Lấy Base/Rider

In [61]:
policy_kpi = individual_kpi
policy_kpi = policy_kpi[['Agent_Number', 'Base', 'Rider']]
policy_kpi = pd.merge(structure, policy_kpi, left_on = 'AgentNumber', right_on ='Agent_Number', how='left')
for i in ['Base', 'Rider']:
    policy_kpi[i] = policy_kpi[i].astype('int')
policy_kpi

,Area_Name,Sales_Unit_Code,Sales_Unit,GM_Whole_Code,GM_Whole_Name,GM_IndCode,GM_IndName,GM_Code,GM_Name,RM_Whole_Code,...,TERRITORY,Leader,Parent_Supervisor_Code,Supervisor_Code,CUTOFF,Date_On_Set,Start_Month,Agent_Number,Base,Rider
0,TAA,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,0,60030655,60030650,60030655,20230131,2023-01-31,2023-01-01,60060890,0,0
1,ABT,AD786,Nguyễn Thị Tố Quyên,0,0,0,0,0,0,60055383,...,0,60055914,60055833,60055914,20230131,2023-01-31,2023-01-01,60060917,0,0
2,EAK,AD174,Thiều Quang Mạnh,0,0,0,0,0,0,0,...,0,60008686,60008686,60019636,20230131,2023-01-31,2023-01-01,60021398,0,0
3,AVP,AD811,Văn Hương Giang,0,0,0,0,0,0,0,...,0,60054777,,60054777,20230131,2023-01-31,2023-01-01,60055999,0,0
4,LAK,AD501,Đặng Văn Phan,0,0,0,0,0,0,60021241,...,0,60075163,60062231,60075163,20230131,2023-01-31,2023-01-01,60075691,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75901,SG3,AD246,Nguyễn Thanh Lộc,0,0,0,0,0,0,0,...,0,0,0,,20230131,2023-01-31,2023-01-01,60039816,0,0
75902,CLA,AD285,Nguyễn Ánh Tuyết,0,0,0,0,0,0,0,...,0,0,0,,20230131,2023-01-31,2023-01-01,60045618,0,0
75903,VIN,AD158,Nguyễn Thị Nguyệt,0,0,0,0,0,0,0,...,0,0,0,,20230131,2023-01-31,2023-01-01,60027162,0,0
75904,SG1,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,0,0,0,,20230131,2023-01-31,2023-01-01,60025232,0,0


In [62]:
#FM:
FM_policy = policy_kpi[['FM_Code', 'AgentNumber', 'Base', 'Rider']]
FM_policy = FM_policy[FM_policy['FM_Code'] != '0']
FM_policy = FM_policy.groupby(['FM_Code'])[['Base','Rider']].sum().reset_index().rename(columns={'FM_Code':'AL_Code'})
FM_policy

,AL_Code,Base,Rider
0,60000014,0,0
1,60000037,0,0
2,60000055,0,0
3,60000100,0,0
4,60000101,0,0
...,...,...,...
8057,60076154,2,2
8058,60076155,2,9
8059,60076158,0,0
8060,60076161,0,0


In [63]:
#DM:
DM_policy = policy_kpi[['DM_Code', 'AgentNumber', 'Base', 'Rider']]
DM_policy = DM_policy[DM_policy['DM_Code'] != '0']
DM_policy = DM_policy.groupby(['DM_Code'])[['Base','Rider']].sum().reset_index().rename(columns={'DM_Code':'AL_Code'})
DM_policy

,AL_Code,Base,Rider
0,60000092,0,0
1,60000231,0,0
2,60000256,0,0
3,60000263,0,0
4,60000459,0,0
...,...,...,...
2157,60076129,8,11
2158,60076133,0,0
2159,60076138,0,0
2160,60118829,1,6


In [64]:
#RM:
RM_policy = policy_kpi[['RM_Code', 'AgentNumber', 'Base', 'Rider']]
RM_policy = RM_policy[RM_policy['RM_Code'] != '0']
RM_policy = RM_policy.groupby(['RM_Code'])[['Base','Rider']].sum().reset_index().rename(columns={'RM_Code':'AL_Code'})
RM_policy

,AL_Code,Base,Rider
0,60000727,8,22
1,60005252,21,66
2,60012146,1,3
3,60012256,6,26
4,60014549,3,10
...,...,...,...
73,60058061,8,30
74,60059003,20,46
75,60060164,0,0
76,60061923,26,56


In [65]:
#GM:
GM_policy = policy_kpi[['GM_Code', 'AgentNumber', 'Base', 'Rider']]
GM_policy = GM_policy[GM_policy['GM_Code'] != '0']
GM_policy = GM_policy.groupby(['GM_Code'])[['Base','Rider']].sum().reset_index().rename(columns={'GM_Code':'AL_Code'})
GM_policy

,AL_Code,Base,Rider
0,60011794,7,32
1,60023377,0,0
2,60023547,2,4


In [95]:
#Tổng hợp kết quả
al_policy = pd.concat([FM_policy, DM_policy, RM_policy, GM_policy])
al_policy = pd.merge(al_cc, al_policy, left_on = 'AgentNumber', right_on ='AL_Code', how='left')
for col in ['Base', 'Rider']:
    al_policy[col] = al_policy[col].astype('int')
al_policy['%Rider/Base'] = np.where(al_policy['Rider'] == 0, 0, round(al_policy['Rider']/al_policy['Base'], 4))
al_policy = al_policy.loc[:, ~al_policy.columns.isin(['AL_Code'])]
al_policy

,Period,AgentNumber,AgentName,AgentGrade,Agent_Status,Date_Appointed,PY2,GM,RM,DM,...,AL Ter,FC Ter,Total Agent Ter,%Ter,CC(+),CC(-),CC,Base,Rider,%Rider/Base
0,202301,60054915,MAI THẾ TRUNG,FM,Enforce,2021-12-24,0.0000,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
1,202301,60006025,NGUYỄN THỊ HẠNH,FM,Terminated,2016-09-26,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
2,202301,60013413,NGÔ THỦY TRIỀU,FM,Terminated,2018-03-26,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
3,202301,60056899,DUMMY HỒ THỊ LONG,FM,Terminated,2022-03-03,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
4,202301,60037327,DUMMY 37327,FM,Terminated,2021-01-07,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10301,202301,60038668,LƯƠNG THỊ TƯỜNG VI,DM,Enforce,2021-03-03,0.0699,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
10302,202301,60033477,MÃ THÀNH HIẾU,DM,Enforce,2022-01-25,0.6428,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
10303,202301,60045450,LƯƠNG ANH QUÂN,FM,Enforce,2021-06-23,0.0000,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0
10304,202301,60008668,NGUYỄN THỊ SÁU,DM,Terminated,2017-06-23,0.0000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0.0


<h3> 8. Tính Active Agent

In [92]:
aa_al = cc_kpi[cc_kpi['CC'] > 0]
aa_al['Active Agent'] = 1
aa_al

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_14556\1599093293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa_al['Active Agent'] = 1


,Area_Name,Sales_Unit_Code,Sales_Unit,GM_Whole_Code,GM_Whole_Name,GM_IndCode,GM_IndName,GM_Code,GM_Name,RM_Whole_Code,...,Parent_Supervisor_Code,Supervisor_Code,CUTOFF,Date_On_Set,Start_Month,Agent_Number,CC(+),CC(-),CC,Active Agent
75,SG1,AD623,Huỳnh Phạm Đăng Khánh,0,0,0,0,0,0,60005252,...,60005252,60055932,20230131,2023-01-31,2023-01-01,60055933,5,0,5,1
239,AHP,AD930,AG Bùi Thị Vân Anh,0,0,0,0,0,0,0,...,60074746,60074836,20230131,2023-01-31,2023-01-01,60074745,1,0,1,1
285,SG1,AD752,Lê Nguyễn Ánh Nguyệt,0,0,0,0,0,0,60060164,...,60070913,60070705,20230131,2023-01-31,2023-01-01,60072004,4,0,4,1
348,DN1,AD668,Lê Trần Tôn Nữ,0,0,0,0,0,0,0,...,60058551,60058552,20230131,2023-01-31,2023-01-01,60058929,2,0,2,1
373,SG1,AD752,Lê Nguyễn Ánh Nguyệt,0,0,0,0,0,0,60060164,...,60062462,60062463,20230131,2023-01-31,2023-01-01,60062475,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74638,DTH,AD918,Đinh Minh Phương,0,0,0,0,0,0,0,...,0,,20230131,2023-01-31,2023-01-01,60072718,2,0,2,1
75287,SG3,AD951,Lê Quang Anh Vũ,0,0,0,0,0,0,0,...,0,,20230131,2023-01-31,2023-01-01,60074756,3,0,3,1
75646,BH2,AD781,Vũ Thị Thanh Hương,0,0,0,0,0,0,60039194,...,0,,20230131,2023-01-31,2023-01-01,60039194,1,0,1,1
75749,DGH,AD988,Nguyễn Hoàng Hiệp,0,0,0,0,0,0,0,...,0,,20230131,2023-01-31,2023-01-01,60075971,1,0,1,1


In [93]:
#FM:
FM_aa = aa_al[['FM_Code', 'Agent_Number', 'Active Agent']]
FM_aa = FM_aa[FM_aa['FM_Code'] !='0']
FM_aa = FM_aa.groupby(['FM_Code'])['Active Agent'].sum().reset_index().rename(columns={'FM_Code':'AL_Code'})
FM_aa

,AL_Code,Active Agent
0,60000482,1
1,60000717,1
2,60005897,1
3,60006677,1
4,60008296,1
...,...,...
416,60076113,1
417,60076119,3
418,60076126,1
419,60076154,1


In [85]:
#DM:
DM_aa = aa_al[['DM_Code', 'Agent_Number', 'Active Agent']]
DM_aa = DM_aa[DM_aa['DM_Code'] !='0']
DM_aa = DM_aa.groupby(['DM_Code'])['Active Agent'].sum().reset_index().rename(columns={'DM_Code':'AL_Code'})
DM_aa

,AL_Code,Active Agent
0,60001255,1
1,60006365,2
2,60006894,1
3,60007708,1
4,60009490,4
...,...,...
316,60075990,1
317,60076083,1
318,60076107,4
319,60076129,6


In [86]:
#RM:
RM_aa = aa_al[['RM_Code', 'Agent_Number', 'Active Agent']]
RM_aa = RM_aa[RM_aa['RM_Code'] !='0']
RM_aa = RM_aa.groupby(['RM_Code'])['Active Agent'].sum().reset_index().rename(columns={'RM_Code':'AL_Code'})
RM_aa

,AL_Code,Active Agent
0,60000727,6
1,60005252,14
2,60012146,1
3,60012256,5
4,60014549,3
5,60014821,4
6,60014874,5
7,60020362,1
8,60020387,2
9,60022528,4


In [87]:
#GM:
GM_aa = aa_al[['GM_Code', 'Agent_Number', 'Active Agent']]
GM_aa = GM_aa[GM_aa['GM_Code'] !='0']
GM_aa = GM_aa.groupby(['GM_Code'])['Active Agent'].sum().reset_index().rename(columns={'GM_Code':'AL_Code'})
GM_aa

,AL_Code,Active Agent
0,60011794,6
1,60023547,1


In [88]:
#Tổng hợp:
sum_aa = pd.concat([FM_aa, DM_aa, RM_aa, GM_aa])
sum_aa

,AL_Code,Active Agent
0,60000482,1
1,60000717,1
2,60005897,1
3,60006677,1
4,60008296,1
...,...,...
38,60058061,3
39,60059003,14
40,60061923,14
0,60011794,6


In [103]:
al_aa_stage = pd.merge(al_policy, sum_aa, left_on = 'AgentNumber', right_on ='AL_Code', how ='left')
al_aa_stage = al_aa_stage.loc[:, ~al_aa_stage.columns.isin(['AL_Code'])]
al_aa_stage['Active Agent'].fillna(value =0, inplace= True)
al_aa_stage['Activity Ratio'] = np.where(al_aa_stage['MP_Total']  == 0, 0,
    round(al_aa_stage['Active Agent'] / al_aa_stage['MP_Total'], 4))
al_aa_stage

,Period,AgentNumber,AgentName,AgentGrade,Agent_Status,Date_Appointed,PY2,GM,RM,DM,...,Total Agent Ter,%Ter,CC(+),CC(-),CC,Base,Rider,%Rider/Base,Active Agent,Activity Ratio
0,202301,60054915,MAI THẾ TRUNG,FM,Enforce,2021-12-24,0.0000,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0.0,0.0,0.0
1,202301,60006025,NGUYỄN THỊ HẠNH,FM,Terminated,2016-09-26,0.0000,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0.0,0.0,NaN
2,202301,60013413,NGÔ THỦY TRIỀU,FM,Terminated,2018-03-26,0.0000,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0.0,0.0,NaN
3,202301,60056899,DUMMY HỒ THỊ LONG,FM,Terminated,2022-03-03,0.0000,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0.0,0.0,NaN
4,202301,60037327,DUMMY 37327,FM,Terminated,2021-01-07,0.0000,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10301,202301,60038668,LƯƠNG THỊ TƯỜNG VI,DM,Enforce,2021-03-03,0.0699,0.0,0.0,1.0,...,NaN,NaN,0,0,0,0,0,0.0,0.0,0.0
10302,202301,60033477,MÃ THÀNH HIẾU,DM,Enforce,2022-01-25,0.6428,0.0,0.0,1.0,...,NaN,NaN,0,0,0,0,0,0.0,0.0,0.0
10303,202301,60045450,LƯƠNG ANH QUÂN,FM,Enforce,2021-06-23,0.0000,0.0,0.0,0.0,...,NaN,NaN,0,0,0,0,0,0.0,0.0,0.0
10304,202301,60008668,NGUYỄN THỊ SÁU,DM,Terminated,2017-06-23,0.0000,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0.0,0.0,NaN


<h3> 9. Tính AFYP và FYP

In [126]:
afyp_aa = individual_kpi[['Agent_Number', 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']]
afyp_aa = pd.merge(structure, afyp_aa, left_on = 'AgentNumber', right_on = 'Agent_Number', how = 'left')
afyp_aa

,Area_Name,Sales_Unit_Code,Sales_Unit,GM_Whole_Code,GM_Whole_Name,GM_IndCode,GM_IndName,GM_Code,GM_Name,RM_Whole_Code,...,Parent_Supervisor_Code,Supervisor_Code,CUTOFF,Date_On_Set,Start_Month,Agent_Number,AFYP(+),AFYP(-),FYP(+),FYP(-)
0,TAA,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,60030650,60030655,20230131,2023-01-31,2023-01-01,60060890,0,0,0,0
1,ABT,AD786,Nguyễn Thị Tố Quyên,0,0,0,0,0,0,60055383,...,60055833,60055914,20230131,2023-01-31,2023-01-01,60060917,0,0,0,0
2,EAK,AD174,Thiều Quang Mạnh,0,0,0,0,0,0,0,...,60008686,60019636,20230131,2023-01-31,2023-01-01,60021398,0,0,0,0
3,AVP,AD811,Văn Hương Giang,0,0,0,0,0,0,0,...,,60054777,20230131,2023-01-31,2023-01-01,60055999,0,0,0,0
4,LAK,AD501,Đặng Văn Phan,0,0,0,0,0,0,60021241,...,60062231,60075163,20230131,2023-01-31,2023-01-01,60075691,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75901,SG3,AD246,Nguyễn Thanh Lộc,0,0,0,0,0,0,0,...,0,,20230131,2023-01-31,2023-01-01,60039816,0,0,0,0
75902,CLA,AD285,Nguyễn Ánh Tuyết,0,0,0,0,0,0,0,...,0,,20230131,2023-01-31,2023-01-01,60045618,0,0,0,0
75903,VIN,AD158,Nguyễn Thị Nguyệt,0,0,0,0,0,0,0,...,0,,20230131,2023-01-31,2023-01-01,60027162,0,0,0,0
75904,SG1,AD082,Ngô Anh Tài,0,0,0,0,0,0,0,...,0,,20230131,2023-01-31,2023-01-01,60025232,0,0,0,0


In [127]:
#FM:
FM_afyp = afyp_aa[['FM_Code', 'Agent_Number', 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']]
FM_afyp = FM_afyp[FM_afyp['FM_Code'] != '0']
for col in ['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']:
    FM_afyp[col] = FM_afyp[col].astype('float')
FM_afyp = FM_afyp.groupby(['FM_Code'])[['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']].sum().reset_index().rename(columns={'FM_Code': 'AL_Code'})
FM_afyp

,AL_Code,AFYP(+),AFYP(-),FYP(+),FYP(-)
0,60000014,0.0,0.0,0.0,0.0
1,60000037,0.0,0.0,0.0,0.0
2,60000055,0.0,0.0,0.0,0.0
3,60000100,0.0,0.0,0.0,0.0
4,60000101,0.0,0.0,0.0,0.0
...,...,...,...,...,...
8057,60076154,30256000.0,0.0,30256000.0,0.0
8058,60076155,32048000.0,0.0,32048000.0,0.0
8059,60076158,0.0,0.0,0.0,0.0
8060,60076161,0.0,0.0,0.0,0.0


In [129]:
#DM:
DM_afyp = afyp_aa[['DM_Code', 'Agent_Number', 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']]
DM_afyp = DM_afyp[DM_afyp['DM_Code'] != '0']
for col in ['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']:
    DM_afyp[col] = DM_afyp[col].astype('float')
DM_afyp = DM_afyp.groupby(['DM_Code'])[['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']].sum().reset_index().rename(columns={'DM_Code': 'AL_Code'})
DM_afyp

,AL_Code,AFYP(+),AFYP(-),FYP(+),FYP(-)
0,60000092,0.0,0.0,0.0,0.0
1,60000231,0.0,0.0,0.0,0.0
2,60000256,0.0,0.0,0.0,0.0
3,60000263,0.0,0.0,0.0,0.0
4,60000459,0.0,0.0,0.0,0.0
...,...,...,...,...,...
2157,60076129,156097000.0,0.0,138093000.0,0.0
2158,60076133,0.0,0.0,0.0,0.0
2159,60076138,0.0,0.0,0.0,0.0
2160,60118829,12674000.0,0.0,6337000.0,0.0


In [130]:
#RM:
RM_afyp = afyp_aa[['RM_Code', 'Agent_Number', 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']]
RM_afyp = RM_afyp[RM_afyp['RM_Code'] != '0']
for col in ['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']:
    RM_afyp[col] = RM_afyp[col].astype('float')
RM_afyp = RM_afyp.groupby(['RM_Code'])[['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']].sum().reset_index().rename(columns={'RM_Code': 'AL_Code'})
RM_afyp

,AL_Code,AFYP(+),AFYP(-),FYP(+),FYP(-)
0,60000727,135211000.0,0.0,142671000.0,0.0
1,60005252,510664000.0,-28216000.0,519719000.0,-28216000.0
2,60012146,15018000.0,0.0,16014200.0,0.0
3,60012256,82618000.0,-33250000.0,84693800.0,-33250000.0
4,60014549,34000000.0,-35000000.0,28128300.0,-35000000.0
...,...,...,...,...,...
73,60058061,120473000.0,-15456000.0,122305000.0,-15456000.0
74,60059003,450385000.0,0.0,450385000.0,0.0
75,60060164,0.0,0.0,5022000.0,0.0
76,60061923,760350100.0,0.0,760370200.0,0.0


In [131]:
#GM:
GM_afyp = afyp_aa[['GM_Code', 'Agent_Number', 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']]
GM_afyp = GM_afyp[GM_afyp['GM_Code'] != '0']
for col in ['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']:
    GM_afyp[col] = GM_afyp[col].astype('float')
GM_afyp = GM_afyp.groupby(['GM_Code'])[['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']].sum().reset_index().rename(columns={'GM_Code': 'AL_Code'})
GM_afyp

,AL_Code,AFYP(+),AFYP(-),FYP(+),FYP(-)
0,60011794,98317000.0,-33250000.0,100679500.0,-33250000.0
1,60023377,0.0,0.0,0.0,0.0
2,60023547,21025000.0,-190078000.0,21042100.0,-190078000.0


In [132]:
sum_afyp = pd.concat([FM_afyp, DM_afyp, RM_afyp, GM_afyp])
sum_afyp

,AL_Code,AFYP(+),AFYP(-),FYP(+),FYP(-)
0,60000014,0.0,0.0,0.0,0.0
1,60000037,0.0,0.0,0.0,0.0
2,60000055,0.0,0.0,0.0,0.0
3,60000100,0.0,0.0,0.0,0.0
4,60000101,0.0,0.0,0.0,0.0
...,...,...,...,...,...
76,60061923,760350100.0,0.0,760370200.0,0.0
77,60071896,0.0,0.0,0.0,0.0
0,60011794,98317000.0,-33250000.0,100679500.0,-33250000.0
1,60023377,0.0,0.0,0.0,0.0


In [176]:
afyp_stage = pd.merge(al_aa_stage, sum_afyp, left_on ='AgentNumber', right_on = 'AL_Code', how='left')
afyp_stage = afyp_stage.loc[:, ~afyp_stage.columns.isin(['AL_Code'])]
afyp_stage['AFYP'] = afyp_stage['AFYP(+)'] + afyp_stage['AFYP(-)']
afyp_stage['FYP'] = afyp_stage['FYP(+)'] + afyp_stage['FYP(-)']
afyp_stage

,Period,AgentNumber,AgentName,AgentGrade,Agent_Status,Date_Appointed,PY2,GM,RM,DM,...,Rider,%Rider/Base,Active Agent,Activity Ratio,AFYP(+),AFYP(-),FYP(+),FYP(-),AFYP,FYP
0,202301,60054915,MAI THẾ TRUNG,FM,Enforce,2021-12-24,0.0000,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,202301,60006025,NGUYỄN THỊ HẠNH,FM,Terminated,2016-09-26,0.0000,NaN,NaN,NaN,...,0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,202301,60013413,NGÔ THỦY TRIỀU,FM,Terminated,2018-03-26,0.0000,NaN,NaN,NaN,...,0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,202301,60056899,DUMMY HỒ THỊ LONG,FM,Terminated,2022-03-03,0.0000,NaN,NaN,NaN,...,0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,202301,60037327,DUMMY 37327,FM,Terminated,2021-01-07,0.0000,NaN,NaN,NaN,...,0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10301,202301,60038668,LƯƠNG THỊ TƯỜNG VI,DM,Enforce,2021-03-03,0.0699,0.0,0.0,1.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10302,202301,60033477,MÃ THÀNH HIẾU,DM,Enforce,2022-01-25,0.6428,0.0,0.0,1.0,...,0,0.0,0.0,0.0,0.0,0.0,6599000.0,0.0,0.0,6599000.0
10303,202301,60045450,LƯƠNG ANH QUÂN,FM,Enforce,2021-06-23,0.0000,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10304,202301,60008668,NGUYỄN THỊ SÁU,DM,Terminated,2017-06-23,0.0000,NaN,NaN,NaN,...,0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [177]:
case_size = afyp_stage
case_size['Case Size'] = np.where(case_size['CC'] == 0, 0, round(case_size['AFYP'] / case_size['CC'], 2))
case_size['Cases_per_Active'] = np.where(case_size['Active Agent'] ==0, 0, 
                                         round(case_size['CC']/ case_size['Active Agent'], 2))
for col in ['GM','RM', 'DM', 'FM', 'AL', 'FC', 'MP_Total', 'AL Trainee', 'Agent Trainee', 'SFC', 'Recruit_RM', 'Recruit_DM', 'Recruit_FM',
 'Recruit AL', 'Recruit FC', 'Recruit Total', 'Recruit Active', 'ActiveRatioNewRecruit', 'AL Ter', 'FC Ter', 'Total Agent Ter',
 '%Ter', 'CC(+)', 'CC(-)', 'CC', 'Base', 'Rider', '%Rider/Base', 'Active Agent', 'Activity Ratio',
 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)', 'AFYP', 'FYP', 'Case Size', 'Cases_per_Active']:
    case_size[col] = case_size[col].astype('float')
case_size.fillna(value=0, inplace= True)
case_size

,Period,AgentNumber,AgentName,AgentGrade,Agent_Status,Date_Appointed,PY2,GM,RM,DM,...,Active Agent,Activity Ratio,AFYP(+),AFYP(-),FYP(+),FYP(-),AFYP,FYP,Case Size,Cases_per_Active
0,202301,60054915,MAI THẾ TRUNG,FM,Enforce,2021-12-24,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,202301,60006025,NGUYỄN THỊ HẠNH,FM,Terminated,2016-09-26,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,202301,60013413,NGÔ THỦY TRIỀU,FM,Terminated,2018-03-26,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,202301,60056899,DUMMY HỒ THỊ LONG,FM,Terminated,2022-03-03,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,202301,60037327,DUMMY 37327,FM,Terminated,2021-01-07,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10301,202301,60038668,LƯƠNG THỊ TƯỜNG VI,DM,Enforce,2021-03-03,0.0699,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10302,202301,60033477,MÃ THÀNH HIẾU,DM,Enforce,2022-01-25,0.6428,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,6599000.0,0.0,0.0,6599000.0,0.0,0.0
10303,202301,60045450,LƯƠNG ANH QUÂN,FM,Enforce,2021-06-23,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10304,202301,60008668,NGUYỄN THỊ SÁU,DM,Terminated,2017-06-23,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [178]:
for col in ['GM','RM', 'DM', 'FM', 'AL', 'FC', 'MP_Total', 'AL Trainee', 'Agent Trainee', 'SFC', 'Recruit_RM', 'Recruit_DM', 'Recruit_FM',
 'Recruit AL', 'Recruit FC', 'Recruit Total', 'Recruit Active', 'ActiveRatioNewRecruit', 'AL Ter', 'FC Ter', 'Total Agent Ter',
 '%Ter', 'CC(+)', 'CC(-)', 'CC', 'Base', 'Rider', '%Rider/Base', 'Active Agent', 'Activity Ratio',
 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)', 'AFYP', 'FYP', 'Case Size', 'Cases_per_Active']:
    case_size[col] = case_size[col].astype('str')
    case_size[col] = np.where(case_size[col] == '0.0', '0', case_size[col])
    case_size[col] = [i.replace('.0','') for i in case_size[col]]
case_size

,Period,AgentNumber,AgentName,AgentGrade,Agent_Status,Date_Appointed,PY2,GM,RM,DM,...,Active Agent,Activity Ratio,AFYP(+),AFYP(-),FYP(+),FYP(-),AFYP,FYP,Case Size,Cases_per_Active
0,202301,60054915,MAI THẾ TRUNG,FM,Enforce,2021-12-24,0.0000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,202301,60006025,NGUYỄN THỊ HẠNH,FM,Terminated,2016-09-26,0.0000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,202301,60013413,NGÔ THỦY TRIỀU,FM,Terminated,2018-03-26,0.0000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,202301,60056899,DUMMY HỒ THỊ LONG,FM,Terminated,2022-03-03,0.0000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,202301,60037327,DUMMY 37327,FM,Terminated,2021-01-07,0.0000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10301,202301,60038668,LƯƠNG THỊ TƯỜNG VI,DM,Enforce,2021-03-03,0.0699,0,0,1,...,0,0,0,0,0,0,0,0,0,0
10302,202301,60033477,MÃ THÀNH HIẾU,DM,Enforce,2022-01-25,0.6428,0,0,1,...,0,0,0,0,6599000,0,0,6599000,0,0
10303,202301,60045450,LƯƠNG ANH QUÂN,FM,Enforce,2021-06-23,0.0000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10304,202301,60008668,NGUYỄN THỊ SÁU,DM,Terminated,2017-06-23,0.0000,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
### Tạo ra khung bảng

In [179]:
final_team_kpi = case_size
#final_team_kpi.to_sql("BSP_TAN_WHOLE_TEAM_KPI", conn_UAT, if_exists='replace', index=False)

In [183]:
insert_query = """\
INSERT INTO [DPO].[BSP_TAN_WHOLE_TEAM_KPI] ([Period],[AgentNumber],[AgentName],[AgentGrade],[Agent_Status],[Date_Appointed],
[PY2],[GM],[RM],[DM],[FM],[AL],[FC],[MP_Total],[AL Trainee],[Agent Trainee],[SFC],[Recruit_RM],[Recruit_DM],[Recruit_FM],[Recruit AL],
[Recruit FC],[Recruit Total],[Recruit Active],[ActiveRatioNewRecruit],[AL Ter],[FC Ter],[Total Agent Ter],[%Ter],[CC(+)],
[CC(-)],[CC],[Base],[Rider],[%Rider/Base],[Active Agent],[Activity Ratio],[AFYP(+)],[AFYP(-)],[AFYP],[FYP(+)],[FYP(-)],
[FYP],[Case Size],[Cases_per_Active]
)
      VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
"""

In [185]:
#3.Tạo Connection với SQL
conn_toSQL = pyo.connect(driver='{ODBC Driver 17 for SQL Server}', host = 'PHLHCMHOUPBI01\PBIRS', database = 'PowerBI', trusted_connection = 'yes',
                        ansi=True,autocommit=True) #LƯU Ý: ANSI GIỮ FONT TIẾNG VIỆT
cursor = conn_toSQL.cursor()
for index, row in final_team_kpi.iterrows():
    cursor.execute(insert_query,
row['Period'],
row['AgentNumber'],
row['AgentName'],
row['AgentGrade'],
row['Agent_Status'],
row['Date_Appointed'],
row['PY2'],
row['GM'],
row['RM'],
row['DM'],
row['FM'],
row['AL'],
row['FC'],
row['MP_Total'],
row['AL Trainee'],
row['Agent Trainee'],
row['SFC'],
row['Recruit_RM'],
row['Recruit_DM'],
row['Recruit_FM'],
row['Recruit AL'],
row['Recruit FC'],
row['Recruit Total'],
row['Recruit Active'],
row['ActiveRatioNewRecruit'],
row['AL Ter'],
row['FC Ter'],
row['Total Agent Ter'],
row['%Ter'],
row['CC(+)'],
row['CC(-)'],
row['CC'],
row['Base'],
row['Rider'],
row['%Rider/Base'],
row['Active Agent'],
row['Activity Ratio'],
row['AFYP(+)'],
row['AFYP(-)'],
row['FYP(+)'],
row['FYP(-)'],
row['AFYP'],
row['FYP'],
row['Case Size'],
row['Cases_per_Active']
)
conn_toSQL.commit()
cursor.close()